Tests for GourmandDWH

FactCountyGrowth incremental model

Testing how this model works

In [ ]:
-- first were going to move our data to another table
select *
into dbo.Countygrowth2
from dbo.CountyGrowth

In [2]:
-- now we'll truncate our original table and add our own row into it to test the incremental model
-- also a row for that will create a duplicate IncrementalCompKey
begin TRANSACTION
TRUNCATE TABLE dbo.countygrowth;
insert into dbo.countygrowth(CountyID, EstimationYear, EstimatedPopulation, LastEditedWhen)
VALUES
(2230,2019,1, DATETIMEFROMPARTS(2021,12,06,12,0,0,0)), --duplicate 
(2230,2020,1, DATETIMEFROMPARTS(2021,12,07,12,0,0,0)) --row from a day after
COMMIT transaction

(2 rows affected)

Total execution time: 00:00:00.009

In [ ]:
-- now we can run the our dbt model and see how it changes
select * from CountyGrowth
-- cmd line: dbt run --select FactCountyGrowth+

Below we can see that only one row was added

The row with a LastEditedWhen greater than the latest (max) value recorded in the target table

In [3]:
select * from _Production.FactCountyGrowth
where CountySourceKey=2230

(11 rows affected)

Total execution time: 00:00:00.015

CountySourceKey,EstimationYear,EstimatedPopulation,LastEditedWhen,IncrementalCompKey
2230,2017,50492,2021-12-06 15:02:56.190,2230-2017
2230,2018,50801,2021-12-06 15:02:56.190,2230-2018
2230,2016,50292,2021-12-06 15:02:56.190,2230-2016
2230,2019,50991,2021-12-06 15:02:56.190,2230-2019
2230,2012,49453,2021-12-06 15:02:56.190,2230-2012
2230,2013,49478,2021-12-06 15:02:56.190,2230-2013
2230,2011,49313,2021-12-06 15:02:56.190,2230-2011
2230,2014,49725,2021-12-06 15:02:56.190,2230-2014
2230,2015,50070,2021-12-06 15:02:56.190,2230-2015
2230,2010,49253,2021-12-06 15:02:56.190,2230-2010


FactBusinessHoldings  incremental model

Testing how this model works

This model allows rows to be updated

In [17]:
-- closed dates already added
select * from _Production.FactBusinessHolding

(5 rows affected)

Total execution time: 00:00:00.010

BusinessKey,BusinessSourceKey,BusinessRating,ReviewCount,CloseDate,IncrementalCompKey
1,1,4.5,390,2021-12-06,1-2021-12-06
2,2,4.5,221,2021-12-06,2-2021-12-06
3,3,4.5,104,2021-12-06,3-2021-12-06
4,4,4.5,230,2021-12-06,4-2021-12-06
5,5,5.0,271,2021-12-06,5-2021-12-06


In [19]:
-- original source table
select * from BusinessHolding

(5 rows affected)

Total execution time: 00:00:00.009

BusinessID,BusinessRating,ReviewCount,CloseDate,IncrementalCompKey
1,4.5,390,2021-12-06,1-2021-12-06
2,4.5,221,2021-12-06,2-2021-12-06
3,4.5,104,2021-12-06,3-2021-12-06
4,4.5,230,2021-12-06,4-2021-12-06
5,5.0,271,2021-12-06,5-2021-12-06


In [20]:
-- first were going to move our data to another table
select *
into dbo.BusinessHolding2
from dbo.BusinessHolding

(5 rows affected)

Total execution time: 00:00:00.006

In [21]:
-- now we'll truncate our original table and add our own row into it to test the incremental model
-- also a row for that will create a duplicate IncrementalCompKey
begin TRANSACTION
TRUNCATE TABLE dbo.BusinessHolding;
insert into dbo.BusinessHolding(BusinessID, BusinessRating, ReviewCount, CloseDate)
VALUES
(5,4.9,272, DATEFROMPARTS(2021,12,06)), --duplicate IncrementalCompKey to force update
(4,4.9,272, DATEFROMPARTS(2021,12,07)) --row from a day after
COMMIT transaction

(2 rows affected)

Total execution time: 00:00:00.055

In [22]:
-- new source table
select * from BusinessHolding
-- now we can run the our dbt model and see how it changes
-- cmd line: dbt run --select FactBusinessHolding+

(2 rows affected)

Total execution time: 00:00:00.019

BusinessID,BusinessRating,ReviewCount,CloseDate,IncrementalCompKey
5,4.9,272,2021-12-06,5-2021-12-06
4,4.9,272,2021-12-07,4-2021-12-07


Here we can see that running the same model twice in one day can allow us to update values

Business #5 had it's values modified from the original 

Also the normal behavior of incremental model is kept as another date is shown for business #4

In [23]:
select * from _Production.FactBusinessHolding

(6 rows affected)

Total execution time: 00:00:00.016

BusinessKey,BusinessSourceKey,BusinessRating,ReviewCount,CloseDate,IncrementalCompKey
1,1,4.5,390,2021-12-06,1-2021-12-06
2,2,4.5,221,2021-12-06,2-2021-12-06
3,3,4.5,104,2021-12-06,3-2021-12-06
4,4,4.5,230,2021-12-06,4-2021-12-06
4,4,4.9,272,2021-12-07,4-2021-12-07
5,5,4.9,272,2021-12-06,5-2021-12-06


DimUser Snapshot model (SCD2)

Testing how this model works

In [24]:
-- the current User dimension with their effective dates
select * from _Production.DimUser

(5 rows affected)

Total execution time: 00:00:00.006

UserKey,UserSourceKey,BusinessCategoryName,UserImageURL,FirstName,LastNameInitial,ValidFrom,ValidTo,is_current
1,1,https://www.yelp.com/user_details?userid=FXiQJQWYckxRwpj7S_SwKg,https://s3-media1.fl.yelpcdn.com/photo/abE-VGsHo71RQyIa1nYGMg/o.jpg,john,H,2021-12-04 14:20:58.420,9999-12-31 00:00:00.000,1
2,2,https://www.yelp.com/user_details?userid=gZKrQxAG2cpsXpwOMLvcow,https://s3-media2.fl.yelpcdn.com/photo/acv6Ew4dAxMLVakJpGQ7mw/o.jpg,Jenn,S,2021-12-04 14:20:58.420,9999-12-31 00:00:00.000,1
3,3,https://www.yelp.com/user_details?userid=OFLo8-ghYJHHzBPS1lDnGg,https://s3-media4.fl.yelpcdn.com/photo/XFWftttmmalK2C5ypyiAdA/o.jpg,Brandon,L,2021-12-04 14:20:58.420,9999-12-31 00:00:00.000,1
4,4,https://www.yelp.com/user_details?userid=so-gR4ril6M4YUpW3gJpIg,https://s3-media3.fl.yelpcdn.com/photo/S-evdfcXnZRYQkUOnBHDbQ/o.jpg,Amanda,K,2021-12-04 14:20:58.420,9999-12-31 00:00:00.000,1
5,5,https://www.yelp.com/user_details?userid=u7cjZvM9tQfDTSHQilAcBA,https://s3-media1.fl.yelpcdn.com/photo/q--tcHd6Cclo7VCAOR-1_A/o.jpg,Paige,L,2021-12-04 14:20:58.420,9999-12-31 00:00:00.000,1


In [ ]:
-- copying data to another table first
select *
into User2
from [User]

In [25]:
-- a look at our user source table prior to insert
select * from [User]

(5 rows affected)

Total execution time: 00:00:00.007

UserID,UserProfileURL,UserImageURL,FirstName,LastNameInitial,LastEditedWhen
1,https://www.yelp.com/user_details?userid=FXiQJQWYckxRwpj7S_SwKg,https://s3-media1.fl.yelpcdn.com/photo/abE-VGsHo71RQyIa1nYGMg/o.jpg,john,H,2021-12-04 14:20:58.420
2,https://www.yelp.com/user_details?userid=gZKrQxAG2cpsXpwOMLvcow,https://s3-media2.fl.yelpcdn.com/photo/acv6Ew4dAxMLVakJpGQ7mw/o.jpg,Jenn,S,2021-12-04 14:20:58.420
3,https://www.yelp.com/user_details?userid=OFLo8-ghYJHHzBPS1lDnGg,https://s3-media4.fl.yelpcdn.com/photo/XFWftttmmalK2C5ypyiAdA/o.jpg,Brandon,L,2021-12-04 14:20:58.420
4,https://www.yelp.com/user_details?userid=so-gR4ril6M4YUpW3gJpIg,https://s3-media3.fl.yelpcdn.com/photo/S-evdfcXnZRYQkUOnBHDbQ/o.jpg,Amanda,K,2021-12-04 14:20:58.420
5,https://www.yelp.com/user_details?userid=u7cjZvM9tQfDTSHQilAcBA,https://s3-media1.fl.yelpcdn.com/photo/q--tcHd6Cclo7VCAOR-1_A/o.jpg,Paige,L,2021-12-04 14:20:58.420


In [26]:
-- now we'll truncate our original table and add our own user edit into it to test the snapshot model
begin TRANSACTION
TRUNCATE TABLE dbo.[User];
insert into dbo.[User](UserID, UserProfileURL, UserImageURL, FirstName, LastNameInitial, LastEditedWhen)
VALUES
(5,CAST('https://www.raindata.xyx/login' as nvarchar),NULL, N'Ronald','D', DATETIMEFROMPARTS(2021,12,06,0,0,0,0)) --changing a user's info
COMMIT transaction

(1 row affected)

Total execution time: 00:00:00.003

In [27]:
-- a look at our user source table after the insert
select * from [User]
-- now we can run 2 of our dbt model and see how it changes
-- cmd line: dbt snapshot --select snap_User
-- cmd line: dbt run --select DimUser

(1 row affected)

Total execution time: 00:00:00.013

UserID,UserProfileURL,UserImageURL,FirstName,LastNameInitial,LastEditedWhen
5,https://www.raindata.xyx/login,NULL,Ronald,D,2021-12-06 00:00:00.000


In [28]:
select * from Snapshots.snap_User
select * from _Production.DimUser

(5 rows affected)

(5 rows affected)

Total execution time: 00:00:00.036

UserID,UserProfileURL,UserImageURL,FirstName,LastNameInitial,LastEditedWhen,dbt_scd_id,dbt_updated_at,dbt_valid_from,dbt_valid_to
1,https://www.yelp.com/user_details?userid=FXiQJQWYckxRwpj7S_SwKg,https://s3-media1.fl.yelpcdn.com/photo/abE-VGsHo71RQyIa1nYGMg/o.jpg,john,H,2021-12-04 14:20:58.420,55C3EF1BA8DC885B144D47561B74D30C,2021-12-04 14:20:58.420,2021-12-04 14:20:58.420,NULL
2,https://www.yelp.com/user_details?userid=gZKrQxAG2cpsXpwOMLvcow,https://s3-media2.fl.yelpcdn.com/photo/acv6Ew4dAxMLVakJpGQ7mw/o.jpg,Jenn,S,2021-12-04 14:20:58.420,AF0257F97D268D0AC6094BF7927064B0,2021-12-04 14:20:58.420,2021-12-04 14:20:58.420,NULL
3,https://www.yelp.com/user_details?userid=OFLo8-ghYJHHzBPS1lDnGg,https://s3-media4.fl.yelpcdn.com/photo/XFWftttmmalK2C5ypyiAdA/o.jpg,Brandon,L,2021-12-04 14:20:58.420,830F51D91F14567346C2262F77F2C1C6,2021-12-04 14:20:58.420,2021-12-04 14:20:58.420,NULL
4,https://www.yelp.com/user_details?userid=so-gR4ril6M4YUpW3gJpIg,https://s3-media3.fl.yelpcdn.com/photo/S-evdfcXnZRYQkUOnBHDbQ/o.jpg,Amanda,K,2021-12-04 14:20:58.420,17BDB040CF087282622383C50B8993FE,2021-12-04 14:20:58.420,2021-12-04 14:20:58.420,NULL
5,https://www.yelp.com/user_details?userid=u7cjZvM9tQfDTSHQilAcBA,https://s3-media1.fl.yelpcdn.com/photo/q--tcHd6Cclo7VCAOR-1_A/o.jpg,Paige,L,2021-12-04 14:20:58.420,38E75DCFC27162D31F849883BC2340A9,2021-12-04 14:20:58.420,2021-12-04 14:20:58.420,NULL


UserKey,UserSourceKey,BusinessCategoryName,UserImageURL,FirstName,LastNameInitial,ValidFrom,ValidTo,is_current
1,1,https://www.yelp.com/user_details?userid=FXiQJQWYckxRwpj7S_SwKg,https://s3-media1.fl.yelpcdn.com/photo/abE-VGsHo71RQyIa1nYGMg/o.jpg,john,H,2021-12-04 14:20:58.420,9999-12-31 00:00:00.000,1
2,2,https://www.yelp.com/user_details?userid=gZKrQxAG2cpsXpwOMLvcow,https://s3-media2.fl.yelpcdn.com/photo/acv6Ew4dAxMLVakJpGQ7mw/o.jpg,Jenn,S,2021-12-04 14:20:58.420,9999-12-31 00:00:00.000,1
3,3,https://www.yelp.com/user_details?userid=OFLo8-ghYJHHzBPS1lDnGg,https://s3-media4.fl.yelpcdn.com/photo/XFWftttmmalK2C5ypyiAdA/o.jpg,Brandon,L,2021-12-04 14:20:58.420,9999-12-31 00:00:00.000,1
4,4,https://www.yelp.com/user_details?userid=so-gR4ril6M4YUpW3gJpIg,https://s3-media3.fl.yelpcdn.com/photo/S-evdfcXnZRYQkUOnBHDbQ/o.jpg,Amanda,K,2021-12-04 14:20:58.420,9999-12-31 00:00:00.000,1
5,5,https://www.yelp.com/user_details?userid=u7cjZvM9tQfDTSHQilAcBA,https://s3-media1.fl.yelpcdn.com/photo/q--tcHd6Cclo7VCAOR-1_A/o.jpg,Paige,L,2021-12-04 14:20:58.420,9999-12-31 00:00:00.000,1


In [29]:
-- here we can see how the new row was inserted and the previous row was made ineffective
-- but also the other rows have end dates
select * from Snapshots.snap_User

(6 rows affected)

Total execution time: 00:00:00.015

UserID,UserProfileURL,UserImageURL,FirstName,LastNameInitial,LastEditedWhen,dbt_scd_id,dbt_updated_at,dbt_valid_from,dbt_valid_to
5,https://www.raindata.xyx/login,NULL,Ronald,D,2021-12-06 00:00:00.000,14403D80C527A4C28B395B68AB613D8E,2021-12-06 00:00:00.000,2021-12-06 00:00:00.000,NULL
1,https://www.yelp.com/user_details?userid=FXiQJQWYckxRwpj7S_SwKg,https://s3-media1.fl.yelpcdn.com/photo/abE-VGsHo71RQyIa1nYGMg/o.jpg,john,H,2021-12-04 14:20:58.420,55C3EF1BA8DC885B144D47561B74D30C,2021-12-04 14:20:58.420,2021-12-04 14:20:58.420,2021-12-08 13:25:15.723
2,https://www.yelp.com/user_details?userid=gZKrQxAG2cpsXpwOMLvcow,https://s3-media2.fl.yelpcdn.com/photo/acv6Ew4dAxMLVakJpGQ7mw/o.jpg,Jenn,S,2021-12-04 14:20:58.420,AF0257F97D268D0AC6094BF7927064B0,2021-12-04 14:20:58.420,2021-12-04 14:20:58.420,2021-12-08 13:25:15.723
3,https://www.yelp.com/user_details?userid=OFLo8-ghYJHHzBPS1lDnGg,https://s3-media4.fl.yelpcdn.com/photo/XFWftttmmalK2C5ypyiAdA/o.jpg,Brandon,L,2021-12-04 14:20:58.420,830F51D91F14567346C2262F77F2C1C6,2021-12-04 14:20:58.420,2021-12-04 14:20:58.420,2021-12-08 13:25:15.723
4,https://www.yelp.com/user_details?userid=so-gR4ril6M4YUpW3gJpIg,https://s3-media3.fl.yelpcdn.com/photo/S-evdfcXnZRYQkUOnBHDbQ/o.jpg,Amanda,K,2021-12-04 14:20:58.420,17BDB040CF087282622383C50B8993FE,2021-12-04 14:20:58.420,2021-12-04 14:20:58.420,2021-12-08 13:25:15.723
5,https://www.yelp.com/user_details?userid=u7cjZvM9tQfDTSHQilAcBA,https://s3-media1.fl.yelpcdn.com/photo/q--tcHd6Cclo7VCAOR-1_A/o.jpg,Paige,L,2021-12-04 14:20:58.420,38E75DCFC27162D31F849883BC2340A9,2021-12-04 14:20:58.420,2021-12-04 14:20:58.420,2021-12-06 00:00:00.000


An interesting thing happened here , as we have  

```
invalidate_hard_deletes=True

```

then if a user isn't reported from the system then his row will be made ineffective

Note: The downstream model (DimReview) won't be affected as it is joined based on the validfrom and validto dates

In [30]:
select * from _Production.DimUser

(6 rows affected)

Total execution time: 00:00:00.012

UserKey,UserSourceKey,BusinessCategoryName,UserImageURL,FirstName,LastNameInitial,ValidFrom,ValidTo,is_current
1,1,https://www.yelp.com/user_details?userid=FXiQJQWYckxRwpj7S_SwKg,https://s3-media1.fl.yelpcdn.com/photo/abE-VGsHo71RQyIa1nYGMg/o.jpg,john,H,2021-12-04 14:20:58.420,2021-12-08 13:25:15.723,0
2,2,https://www.yelp.com/user_details?userid=gZKrQxAG2cpsXpwOMLvcow,https://s3-media2.fl.yelpcdn.com/photo/acv6Ew4dAxMLVakJpGQ7mw/o.jpg,Jenn,S,2021-12-04 14:20:58.420,2021-12-08 13:25:15.723,0
3,3,https://www.yelp.com/user_details?userid=OFLo8-ghYJHHzBPS1lDnGg,https://s3-media4.fl.yelpcdn.com/photo/XFWftttmmalK2C5ypyiAdA/o.jpg,Brandon,L,2021-12-04 14:20:58.420,2021-12-08 13:25:15.723,0
4,4,https://www.yelp.com/user_details?userid=so-gR4ril6M4YUpW3gJpIg,https://s3-media3.fl.yelpcdn.com/photo/S-evdfcXnZRYQkUOnBHDbQ/o.jpg,Amanda,K,2021-12-04 14:20:58.420,2021-12-08 13:25:15.723,0
5,5,https://www.raindata.xyx/login,NULL,Ronald,D,2021-12-06 00:00:00.000,9999-12-31 00:00:00.000,1
6,5,https://www.yelp.com/user_details?userid=u7cjZvM9tQfDTSHQilAcBA,https://s3-media1.fl.yelpcdn.com/photo/q--tcHd6Cclo7VCAOR-1_A/o.jpg,Paige,L,2021-12-04 14:20:58.420,2021-12-06 00:00:00.000,0


We'll rerun the model with the full source data

also the snapshot table will be deleted to reset it

In [34]:
begin TRANSACTION
TRUNCATE table [User]
insert into [User]
select *
from GourmandOLTP._Production.[User]
COMMIT TRAN

(5 rows affected)

Total execution time: 00:00:00.009

In [35]:
select * from [User]

(5 rows affected)

Total execution time: 00:00:00.012

UserID,UserProfileURL,UserImageURL,FirstName,LastNameInitial,LastEditedWhen
1,https://www.yelp.com/user_details?userid=FXiQJQWYckxRwpj7S_SwKg,https://s3-media1.fl.yelpcdn.com/photo/abE-VGsHo71RQyIa1nYGMg/o.jpg,john,H,2021-12-06 15:02:54.330
2,https://www.yelp.com/user_details?userid=gZKrQxAG2cpsXpwOMLvcow,https://s3-media2.fl.yelpcdn.com/photo/acv6Ew4dAxMLVakJpGQ7mw/o.jpg,Jenn,S,2021-12-06 15:02:54.330
3,https://www.yelp.com/user_details?userid=OFLo8-ghYJHHzBPS1lDnGg,https://s3-media4.fl.yelpcdn.com/photo/XFWftttmmalK2C5ypyiAdA/o.jpg,Brandon,L,2021-12-06 15:02:54.330
4,https://www.yelp.com/user_details?userid=so-gR4ril6M4YUpW3gJpIg,https://s3-media3.fl.yelpcdn.com/photo/S-evdfcXnZRYQkUOnBHDbQ/o.jpg,Amanda,K,2021-12-06 15:02:54.330
5,https://www.yelp.com/user_details?userid=u7cjZvM9tQfDTSHQilAcBA,https://s3-media1.fl.yelpcdn.com/photo/q--tcHd6Cclo7VCAOR-1_A/o.jpg,Paige,L,2021-12-06 15:02:54.330


In [36]:
DROP TABLE Snapshots.snap_User

-- cmd line: dbt snapshot --select snap_User
-- cmd line: dbt run --select DimUser

Commands completed successfully.

Total execution time: 00:00:00.003

Delete this cell and bottom cell...make sure source test includes uniqueness of the source table

In [33]:
select * from Snapshots.snap_User

(6 rows affected)

Total execution time: 00:00:00.019

UserID,UserProfileURL,UserImageURL,FirstName,LastNameInitial,LastEditedWhen,dbt_scd_id,dbt_updated_at,dbt_valid_from,dbt_valid_to
5,https://www.raindata.xyx/login,NULL,Ronald,D,2021-12-06 00:00:00.000,14403D80C527A4C28B395B68AB613D8E,2021-12-06 00:00:00.000,2021-12-06 00:00:00.000,NULL
1,https://www.yelp.com/user_details?userid=FXiQJQWYckxRwpj7S_SwKg,https://s3-media1.fl.yelpcdn.com/photo/abE-VGsHo71RQyIa1nYGMg/o.jpg,john,H,2021-12-06 15:02:54.330,6221ABA86B084602DF8DE7F2D33263E5,2021-12-06 15:02:54.330,2021-12-06 15:02:54.330,NULL
2,https://www.yelp.com/user_details?userid=gZKrQxAG2cpsXpwOMLvcow,https://s3-media2.fl.yelpcdn.com/photo/acv6Ew4dAxMLVakJpGQ7mw/o.jpg,Jenn,S,2021-12-06 15:02:54.330,400D123B610239533F4D5BE6DB626ADA,2021-12-06 15:02:54.330,2021-12-06 15:02:54.330,NULL
3,https://www.yelp.com/user_details?userid=OFLo8-ghYJHHzBPS1lDnGg,https://s3-media4.fl.yelpcdn.com/photo/XFWftttmmalK2C5ypyiAdA/o.jpg,Brandon,L,2021-12-06 15:02:54.330,27121E463B43EE65D6B2229AB0B4152C,2021-12-06 15:02:54.330,2021-12-06 15:02:54.330,NULL
4,https://www.yelp.com/user_details?userid=so-gR4ril6M4YUpW3gJpIg,https://s3-media3.fl.yelpcdn.com/photo/S-evdfcXnZRYQkUOnBHDbQ/o.jpg,Amanda,K,2021-12-06 15:02:54.330,BE3D6925C6017215C1F1604B58C4A837,2021-12-06 15:02:54.330,2021-12-06 15:02:54.330,NULL
5,https://www.yelp.com/user_details?userid=u7cjZvM9tQfDTSHQilAcBA,https://s3-media1.fl.yelpcdn.com/photo/q--tcHd6Cclo7VCAOR-1_A/o.jpg,Paige,L,2021-12-06 15:02:54.330,1F23392E436CA417A8437A8E74D6CF6C,2021-12-06 15:02:54.330,2021-12-06 15:02:54.330,NULL


In [38]:
select * from Snapshots.snap_User
select * from _Production.DimUser

(5 rows affected)

(5 rows affected)

Total execution time: 00:00:00.023

UserID,UserProfileURL,UserImageURL,FirstName,LastNameInitial,LastEditedWhen,dbt_scd_id,dbt_updated_at,dbt_valid_from,dbt_valid_to
1,https://www.yelp.com/user_details?userid=FXiQJQWYckxRwpj7S_SwKg,https://s3-media1.fl.yelpcdn.com/photo/abE-VGsHo71RQyIa1nYGMg/o.jpg,john,H,2021-12-06 15:02:54.330,6221ABA86B084602DF8DE7F2D33263E5,2021-12-06 15:02:54.330,2021-12-06 15:02:54.330,NULL
2,https://www.yelp.com/user_details?userid=gZKrQxAG2cpsXpwOMLvcow,https://s3-media2.fl.yelpcdn.com/photo/acv6Ew4dAxMLVakJpGQ7mw/o.jpg,Jenn,S,2021-12-06 15:02:54.330,400D123B610239533F4D5BE6DB626ADA,2021-12-06 15:02:54.330,2021-12-06 15:02:54.330,NULL
3,https://www.yelp.com/user_details?userid=OFLo8-ghYJHHzBPS1lDnGg,https://s3-media4.fl.yelpcdn.com/photo/XFWftttmmalK2C5ypyiAdA/o.jpg,Brandon,L,2021-12-06 15:02:54.330,27121E463B43EE65D6B2229AB0B4152C,2021-12-06 15:02:54.330,2021-12-06 15:02:54.330,NULL
4,https://www.yelp.com/user_details?userid=so-gR4ril6M4YUpW3gJpIg,https://s3-media3.fl.yelpcdn.com/photo/S-evdfcXnZRYQkUOnBHDbQ/o.jpg,Amanda,K,2021-12-06 15:02:54.330,BE3D6925C6017215C1F1604B58C4A837,2021-12-06 15:02:54.330,2021-12-06 15:02:54.330,NULL
5,https://www.yelp.com/user_details?userid=u7cjZvM9tQfDTSHQilAcBA,https://s3-media1.fl.yelpcdn.com/photo/q--tcHd6Cclo7VCAOR-1_A/o.jpg,Paige,L,2021-12-06 15:02:54.330,1F23392E436CA417A8437A8E74D6CF6C,2021-12-06 15:02:54.330,2021-12-06 15:02:54.330,NULL


UserKey,UserSourceKey,BusinessCategoryName,UserImageURL,FirstName,LastNameInitial,ValidFrom,ValidTo,is_current
1,1,https://www.yelp.com/user_details?userid=FXiQJQWYckxRwpj7S_SwKg,https://s3-media1.fl.yelpcdn.com/photo/abE-VGsHo71RQyIa1nYGMg/o.jpg,john,H,2021-12-06 15:02:54.330,9999-12-31 00:00:00.000,1
2,2,https://www.yelp.com/user_details?userid=gZKrQxAG2cpsXpwOMLvcow,https://s3-media2.fl.yelpcdn.com/photo/acv6Ew4dAxMLVakJpGQ7mw/o.jpg,Jenn,S,2021-12-06 15:02:54.330,9999-12-31 00:00:00.000,1
3,3,https://www.yelp.com/user_details?userid=OFLo8-ghYJHHzBPS1lDnGg,https://s3-media4.fl.yelpcdn.com/photo/XFWftttmmalK2C5ypyiAdA/o.jpg,Brandon,L,2021-12-06 15:02:54.330,9999-12-31 00:00:00.000,1
4,4,https://www.yelp.com/user_details?userid=so-gR4ril6M4YUpW3gJpIg,https://s3-media3.fl.yelpcdn.com/photo/S-evdfcXnZRYQkUOnBHDbQ/o.jpg,Amanda,K,2021-12-06 15:02:54.330,9999-12-31 00:00:00.000,1
5,5,https://www.yelp.com/user_details?userid=u7cjZvM9tQfDTSHQilAcBA,https://s3-media1.fl.yelpcdn.com/photo/q--tcHd6Cclo7VCAOR-1_A/o.jpg,Paige,L,2021-12-06 15:02:54.330,9999-12-31 00:00:00.000,1


In [42]:
delete from [user]
where UserID = 5

(2 rows affected)

Total execution time: 00:00:00.001

In [43]:
-- now we'll add our own user edit into it to test the snapshot model
begin TRANSACTION
insert into dbo.[User](UserID, UserProfileURL, UserImageURL, FirstName, LastNameInitial, LastEditedWhen)
VALUES
(5,CAST('https://www.raindata.xyx/login' as nvarchar),NULL, N'Ronald','D', DATETIMEFROMPARTS(2021,12,06,20,0,0,0)) --changing a user's info
COMMIT TRANSACTION
select * from dbo.[User]

(1 row affected)

(5 rows affected)

Total execution time: 00:00:00.014

UserID,UserProfileURL,UserImageURL,FirstName,LastNameInitial,LastEditedWhen
1,https://www.yelp.com/user_details?userid=FXiQJQWYckxRwpj7S_SwKg,https://s3-media1.fl.yelpcdn.com/photo/abE-VGsHo71RQyIa1nYGMg/o.jpg,john,H,2021-12-06 15:02:54.330
2,https://www.yelp.com/user_details?userid=gZKrQxAG2cpsXpwOMLvcow,https://s3-media2.fl.yelpcdn.com/photo/acv6Ew4dAxMLVakJpGQ7mw/o.jpg,Jenn,S,2021-12-06 15:02:54.330
3,https://www.yelp.com/user_details?userid=OFLo8-ghYJHHzBPS1lDnGg,https://s3-media4.fl.yelpcdn.com/photo/XFWftttmmalK2C5ypyiAdA/o.jpg,Brandon,L,2021-12-06 15:02:54.330
4,https://www.yelp.com/user_details?userid=so-gR4ril6M4YUpW3gJpIg,https://s3-media3.fl.yelpcdn.com/photo/S-evdfcXnZRYQkUOnBHDbQ/o.jpg,Amanda,K,2021-12-06 15:02:54.330
5,https://www.raindata.xyx/login,NULL,Ronald,D,2021-12-06 20:00:00.000


In [ ]:
-- now we'll run this
-- cmd line: dbt snapshot --select snap_User
-- cmd line: dbt run --select DimUser

Here cansee the user information is modified accordingly

In [44]:
select * from Snapshots.snap_User
select * from _Production.DimUser

(6 rows affected)

(6 rows affected)

Total execution time: 00:00:00.077

UserID,UserProfileURL,UserImageURL,FirstName,LastNameInitial,LastEditedWhen,dbt_scd_id,dbt_updated_at,dbt_valid_from,dbt_valid_to
1,https://www.yelp.com/user_details?userid=FXiQJQWYckxRwpj7S_SwKg,https://s3-media1.fl.yelpcdn.com/photo/abE-VGsHo71RQyIa1nYGMg/o.jpg,john,H,2021-12-06 15:02:54.330,6221ABA86B084602DF8DE7F2D33263E5,2021-12-06 15:02:54.330,2021-12-06 15:02:54.330,NULL
2,https://www.yelp.com/user_details?userid=gZKrQxAG2cpsXpwOMLvcow,https://s3-media2.fl.yelpcdn.com/photo/acv6Ew4dAxMLVakJpGQ7mw/o.jpg,Jenn,S,2021-12-06 15:02:54.330,400D123B610239533F4D5BE6DB626ADA,2021-12-06 15:02:54.330,2021-12-06 15:02:54.330,NULL
3,https://www.yelp.com/user_details?userid=OFLo8-ghYJHHzBPS1lDnGg,https://s3-media4.fl.yelpcdn.com/photo/XFWftttmmalK2C5ypyiAdA/o.jpg,Brandon,L,2021-12-06 15:02:54.330,27121E463B43EE65D6B2229AB0B4152C,2021-12-06 15:02:54.330,2021-12-06 15:02:54.330,NULL
4,https://www.yelp.com/user_details?userid=so-gR4ril6M4YUpW3gJpIg,https://s3-media3.fl.yelpcdn.com/photo/S-evdfcXnZRYQkUOnBHDbQ/o.jpg,Amanda,K,2021-12-06 15:02:54.330,BE3D6925C6017215C1F1604B58C4A837,2021-12-06 15:02:54.330,2021-12-06 15:02:54.330,NULL
5,https://www.raindata.xyx/login,NULL,Ronald,D,2021-12-06 20:00:00.000,D647CEEAFC34CABE9CD436101DA15638,2021-12-06 20:00:00.000,2021-12-06 20:00:00.000,NULL
5,https://www.yelp.com/user_details?userid=u7cjZvM9tQfDTSHQilAcBA,https://s3-media1.fl.yelpcdn.com/photo/q--tcHd6Cclo7VCAOR-1_A/o.jpg,Paige,L,2021-12-06 15:02:54.330,1F23392E436CA417A8437A8E74D6CF6C,2021-12-06 15:02:54.330,2021-12-06 15:02:54.330,2021-12-06 20:00:00.000


UserKey,UserSourceKey,BusinessCategoryName,UserImageURL,FirstName,LastNameInitial,ValidFrom,ValidTo,is_current
1,1,https://www.yelp.com/user_details?userid=FXiQJQWYckxRwpj7S_SwKg,https://s3-media1.fl.yelpcdn.com/photo/abE-VGsHo71RQyIa1nYGMg/o.jpg,john,H,2021-12-06 15:02:54.330,9999-12-31 00:00:00.000,1
2,2,https://www.yelp.com/user_details?userid=gZKrQxAG2cpsXpwOMLvcow,https://s3-media2.fl.yelpcdn.com/photo/acv6Ew4dAxMLVakJpGQ7mw/o.jpg,Jenn,S,2021-12-06 15:02:54.330,9999-12-31 00:00:00.000,1
3,3,https://www.yelp.com/user_details?userid=OFLo8-ghYJHHzBPS1lDnGg,https://s3-media4.fl.yelpcdn.com/photo/XFWftttmmalK2C5ypyiAdA/o.jpg,Brandon,L,2021-12-06 15:02:54.330,9999-12-31 00:00:00.000,1
4,4,https://www.yelp.com/user_details?userid=so-gR4ril6M4YUpW3gJpIg,https://s3-media3.fl.yelpcdn.com/photo/S-evdfcXnZRYQkUOnBHDbQ/o.jpg,Amanda,K,2021-12-06 15:02:54.330,9999-12-31 00:00:00.000,1
5,5,https://www.raindata.xyx/login,NULL,Ronald,D,2021-12-06 20:00:00.000,9999-12-31 00:00:00.000,1
6,5,https://www.yelp.com/user_details?userid=u7cjZvM9tQfDTSHQilAcBA,https://s3-media1.fl.yelpcdn.com/photo/q--tcHd6Cclo7VCAOR-1_A/o.jpg,Paige,L,2021-12-06 15:02:54.330,2021-12-06 20:00:00.000,0


before checking review I want to mess around with business

Now we'll experiment with the snapshot model for business

In [45]:
-- here we have the current business dimension (will be identical to staging)
select * from _Production.DimBusiness

(5 rows affected)

Total execution time: 00:00:00.009

BusinessKey,BusinessSourceKey,BusinessName,ChainName,CitySourceKey,PaymentLevelSymbol,PaymentLevelName,Longitude,Latitude,AddressLine1,AddressLine2,AddressLine3,ZipCode,BusinessPhone,BusinessURL,is_closed,DistanceToCounty,ValidFrom,ValidTo,is_current
1,1,central-montgomery-3,Central,1,$$$,High,-86.310560,32.380180,129 Coosa St,NULL,NULL,36104,13345171155,https://www.yelp.com/biz/central-montgomery-3?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,36172,2021-12-06 15:02:55.797,9999-12-31 00:00:00.000,1
2,2,d-road-cafe-montgomery,D' Road Cafe,1,$$,Low,-86.309962,32.376970,121 Montgomery St,NULL,NULL,36104,13343282938,https://www.yelp.com/biz/d-road-cafe-montgomery?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,36365,2021-12-06 15:02:55.797,9999-12-31 00:00:00.000,1
3,3,lÃ¡taste-prattville-2,LÃ¡Taste,2,$$,Low,-86.420700,32.459576,1868 E Main St,NULL,NULL,36066,13343561149,https://www.yelp.com/biz/l%C3%A1taste-prattville-2?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,23567,2021-12-06 15:02:55.797,9999-12-31 00:00:00.000,1
4,4,mrs-b-s-home-cooking-montgomery,Mrs Bâ€™s Home Cooking,1,$$,Low,-86.348587,32.366208,415 Airbase Blvd,NULL,NULL,36104,13342645495,https://www.yelp.com/biz/mrs-b-s-home-cooking-montgomery?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,33610,2021-12-06 15:02:55.797,9999-12-31 00:00:00.000,1
5,5,uncle-micks-cajun-market-and-cafe-prattville,Uncle Mick's Cajun Market & Cafe,2,$$,Low,-86.474090,32.460040,136 W Main St,NULL,NULL,36067,13343611020,https://www.yelp.com/biz/uncle-micks-cajun-market-and-cafe-prattville?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,18704,2021-12-06 15:02:55.797,9999-12-31 00:00:00.000,1


In [46]:
-- here is the snapshot table
select * from Snapshots.snap_business

(5 rows affected)

Total execution time: 00:00:00.423

BusinessID,BusinessName,ChainName,AddressLine1,AddressLine2,AddressLine3,Latitude,Longitude,ZipCode,BusinessPhone,BusinessURL,is_closed,DistanceToCounty,CityID,CountyID,StateID,PaymentLevelID,LastEditedWhen,dbt_scd_id,dbt_updated_at,dbt_valid_from,dbt_valid_to
2,d-road-cafe-montgomery,D' Road Cafe,121 Montgomery St,NULL,NULL,32.376970,-86.309962,36104,13343282938,https://www.yelp.com/biz/d-road-cafe-montgomery?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,36365,1,3102,1,1,2021-12-06 15:02:55.797,400D123B610239533F4D5BE6DB626ADA,2021-12-06 15:02:55.797,2021-12-06 15:02:55.797,NULL
1,central-montgomery-3,Central,129 Coosa St,NULL,NULL,32.380180,-86.310560,36104,13345171155,https://www.yelp.com/biz/central-montgomery-3?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,36172,1,3102,1,2,2021-12-06 15:02:55.797,6221ABA86B084602DF8DE7F2D33263E5,2021-12-06 15:02:55.797,2021-12-06 15:02:55.797,NULL
5,uncle-micks-cajun-market-and-cafe-prattville,Uncle Mick's Cajun Market & Cafe,136 W Main St,NULL,NULL,32.460040,-86.474090,36067,13343611020,https://www.yelp.com/biz/uncle-micks-cajun-market-and-cafe-prattville?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,18704,2,3102,1,1,2021-12-06 15:02:55.797,1F23392E436CA417A8437A8E74D6CF6C,2021-12-06 15:02:55.797,2021-12-06 15:02:55.797,NULL
4,mrs-b-s-home-cooking-montgomery,Mrs Bâ€™s Home Cooking,415 Airbase Blvd,NULL,NULL,32.366208,-86.348587,36104,13342645495,https://www.yelp.com/biz/mrs-b-s-home-cooking-montgomery?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,33610,1,3102,1,1,2021-12-06 15:02:55.797,BE3D6925C6017215C1F1604B58C4A837,2021-12-06 15:02:55.797,2021-12-06 15:02:55.797,NULL
3,lÃ¡taste-prattville-2,LÃ¡Taste,1868 E Main St,NULL,NULL,32.459576,-86.420700,36066,13343561149,https://www.yelp.com/biz/l%C3%A1taste-prattville-2?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,23567,2,3102,1,1,2021-12-06 15:02:55.797,27121E463B43EE65D6B2229AB0B4152C,2021-12-06 15:02:55.797,2021-12-06 15:02:55.797,NULL


In [47]:
-- here is our underlying source table
select * FROM dbo.business

(5 rows affected)

Total execution time: 00:00:00.028

BusinessID,BusinessName,ChainName,AddressLine1,AddressLine2,AddressLine3,Latitude,Longitude,ZipCode,BusinessPhone,BusinessURL,is_closed,DistanceToCounty,CityID,CountyID,StateID,PaymentLevelID,LastEditedWhen
2,d-road-cafe-montgomery,D' Road Cafe,121 Montgomery St,NULL,NULL,32.376970,-86.309962,36104,13343282938,https://www.yelp.com/biz/d-road-cafe-montgomery?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,36365,1,3102,1,1,2021-12-06 15:02:55.797
1,central-montgomery-3,Central,129 Coosa St,NULL,NULL,32.380180,-86.310560,36104,13345171155,https://www.yelp.com/biz/central-montgomery-3?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,36172,1,3102,1,2,2021-12-06 15:02:55.797
5,uncle-micks-cajun-market-and-cafe-prattville,Uncle Mick's Cajun Market & Cafe,136 W Main St,NULL,NULL,32.460040,-86.474090,36067,13343611020,https://www.yelp.com/biz/uncle-micks-cajun-market-and-cafe-prattville?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,18704,2,3102,1,1,2021-12-06 15:02:55.797
4,mrs-b-s-home-cooking-montgomery,Mrs Bâ€™s Home Cooking,415 Airbase Blvd,NULL,NULL,32.366208,-86.348587,36104,13342645495,https://www.yelp.com/biz/mrs-b-s-home-cooking-montgomery?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,33610,1,3102,1,1,2021-12-06 15:02:55.797
3,lÃ¡taste-prattville-2,LÃ¡Taste,1868 E Main St,NULL,NULL,32.459576,-86.420700,36066,13343561149,https://www.yelp.com/biz/l%C3%A1taste-prattville-2?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,23567,2,3102,1,1,2021-12-06 15:02:55.797


In [48]:
-- to begin we'll duplicate our source data into another table
select *
into dbo.business2
from dbo.business

(5 rows affected)

Total execution time: 00:00:00.005

In [49]:
BEGIN TRANSACTION
UPDATE dbo.business
SET is_closed = 1, LastEditedWhen=GETDATE()
WHERE BusinessID=5 
COMMIT TRAN
select * FROM dbo.business

(1 row affected)

(5 rows affected)

Total execution time: 00:00:00.021

BusinessID,BusinessName,ChainName,AddressLine1,AddressLine2,AddressLine3,Latitude,Longitude,ZipCode,BusinessPhone,BusinessURL,is_closed,DistanceToCounty,CityID,CountyID,StateID,PaymentLevelID,LastEditedWhen
2,d-road-cafe-montgomery,D' Road Cafe,121 Montgomery St,NULL,NULL,32.376970,-86.309962,36104,13343282938,https://www.yelp.com/biz/d-road-cafe-montgomery?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,36365,1,3102,1,1,2021-12-06 15:02:55.797
1,central-montgomery-3,Central,129 Coosa St,NULL,NULL,32.380180,-86.310560,36104,13345171155,https://www.yelp.com/biz/central-montgomery-3?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,36172,1,3102,1,2,2021-12-06 15:02:55.797
5,uncle-micks-cajun-market-and-cafe-prattville,Uncle Mick's Cajun Market & Cafe,136 W Main St,NULL,NULL,32.460040,-86.474090,36067,13343611020,https://www.yelp.com/biz/uncle-micks-cajun-market-and-cafe-prattville?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,1,18704,2,3102,1,1,2021-12-08 15:00:52.453
4,mrs-b-s-home-cooking-montgomery,Mrs Bâ€™s Home Cooking,415 Airbase Blvd,NULL,NULL,32.366208,-86.348587,36104,13342645495,https://www.yelp.com/biz/mrs-b-s-home-cooking-montgomery?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,33610,1,3102,1,1,2021-12-06 15:02:55.797
3,lÃ¡taste-prattville-2,LÃ¡Taste,1868 E Main St,NULL,NULL,32.459576,-86.420700,36066,13343561149,https://www.yelp.com/biz/l%C3%A1taste-prattville-2?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,23567,2,3102,1,1,2021-12-06 15:02:55.797


In [51]:
-- now we'll rerun the models with these commands

-- dbt snapshot --select snap_dimbusiness1
-- dbt run --select dimbusiness1
-- dbt run --select DimBusiness

Commands completed successfully.

Total execution time: 00:00:00

In [50]:
-- here we can see the changes
select * from _Production.DimBusiness

(6 rows affected)

Total execution time: 00:00:00.029

BusinessKey,BusinessSourceKey,BusinessName,ChainName,CitySourceKey,PaymentLevelSymbol,PaymentLevelName,Longitude,Latitude,AddressLine1,AddressLine2,AddressLine3,ZipCode,BusinessPhone,BusinessURL,is_closed,DistanceToCounty,ValidFrom,ValidTo,is_current
1,1,central-montgomery-3,Central,1,$$$,High,-86.310560,32.380180,129 Coosa St,NULL,NULL,36104,13345171155,https://www.yelp.com/biz/central-montgomery-3?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,36172,2021-12-06 15:02:55.797,9999-12-31 00:00:00.000,1
2,2,d-road-cafe-montgomery,D' Road Cafe,1,$$,Low,-86.309962,32.376970,121 Montgomery St,NULL,NULL,36104,13343282938,https://www.yelp.com/biz/d-road-cafe-montgomery?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,36365,2021-12-06 15:02:55.797,9999-12-31 00:00:00.000,1
3,3,lÃ¡taste-prattville-2,LÃ¡Taste,2,$$,Low,-86.420700,32.459576,1868 E Main St,NULL,NULL,36066,13343561149,https://www.yelp.com/biz/l%C3%A1taste-prattville-2?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,23567,2021-12-06 15:02:55.797,9999-12-31 00:00:00.000,1
4,4,mrs-b-s-home-cooking-montgomery,Mrs Bâ€™s Home Cooking,1,$$,Low,-86.348587,32.366208,415 Airbase Blvd,NULL,NULL,36104,13342645495,https://www.yelp.com/biz/mrs-b-s-home-cooking-montgomery?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,33610,2021-12-06 15:02:55.797,9999-12-31 00:00:00.000,1
5,5,uncle-micks-cajun-market-and-cafe-prattville,Uncle Mick's Cajun Market & Cafe,2,$$,Low,-86.474090,32.460040,136 W Main St,NULL,NULL,36067,13343611020,https://www.yelp.com/biz/uncle-micks-cajun-market-and-cafe-prattville?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,18704,2021-12-06 15:02:55.797,2021-12-08 15:00:52.453,0
6,5,uncle-micks-cajun-market-and-cafe-prattville,Uncle Mick's Cajun Market & Cafe,2,$$,Low,-86.474090,32.460040,136 W Main St,NULL,NULL,36067,13343611020,https://www.yelp.com/biz/uncle-micks-cajun-market-and-cafe-prattville?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,1,18704,2021-12-08 15:00:52.453,9999-12-31 00:00:00.000,1


Now we'll test our Review model which is affected by both our user dimension and business dimension

we're going to quickly make a review that wil correspond to both the user and business we just edited

In [65]:
-- our current reviews (due to noe matching businesses)
select * from _Production.DimReview

(0 rows affected)

Total execution time: 00:00:00.008

ReviewKey,ReviewSourceKey,ReviewURL,ReviewExtract,ReviewRating,CreatedAt,InsertedAt,UserKey,BusinessKey,ValidFrom,ValidTo,is_current


In [66]:
-- snapshot table
select * from Snapshots.snap_review

(10 rows affected)

Total execution time: 00:00:00.020

ReviewID,ReviewURL,ReviewExtract,ReviewRating,UserID,BusinessID,CreatedAt,InsertedAt,dbt_scd_id,dbt_updated_at,dbt_valid_from,dbt_valid_to
1,https://www.yelp.com/biz/pine-coffee-supply-pinedale?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&hrid=veCFBCoxT2Zg22-1On2nag&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=2FlPnf5brM1FB2UTY6vGnQ,Visited Pinedale on a whim and this place is a little gem! I loved the garage door and murals - it created a very modern ambiance and let in the lovely warm...,5,5,NULL,2021-06-05 16:41:37.000,2021-12-06 16:02:55.317,815880AA9E7400140169211360EC87C9,2021-12-06 16:02:55.317,2021-12-06 16:02:55.317,NULL
2,https://www.yelp.com/biz/pine-coffee-supply-pinedale?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&hrid=kBEtq46Afm_h1pIrNDMbWQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=2FlPnf5brM1FB2UTY6vGnQ,The worst cup of market value coffee I have ever had. Fifty cents for cream. The girls working the counter were too busy talking to serve customers,1,1,NULL,2021-09-03 13:17:37.000,2021-12-06 16:02:55.317,04DB06711F4016E8076E5E3A454B81FF,2021-12-06 16:02:55.317,2021-12-06 16:02:55.317,NULL
3,https://www.yelp.com/biz/pine-coffee-supply-pinedale?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&hrid=Q5-dXXyBhHbQOM89grHWlg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=2FlPnf5brM1FB2UTY6vGnQ,Best coffee in town. Oat-milk latte was a creamy espresso bomb that blew up my taste buds. Super cool shop and decor.,5,3,NULL,2021-10-14 10:47:04.000,2021-12-06 16:02:55.317,CFD45FF4049D52B55B72E755079ACA8E,2021-12-06 16:02:55.317,2021-12-06 16:02:55.317,NULL
4,https://www.yelp.com/biz/turkuaz-grill-riverhead?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&hrid=Gv29EXhnHu5Xe4bvsktiGw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=2FlPnf5brM1FB2UTY6vGnQ,"I punched in ""ethnic eats"" in my trusty Yelp search bar while having a relaxing weekend in the Hamptons and Turkuaz was the first up on the list. Being a...",5,2,NULL,2021-10-15 18:06:41.000,2021-12-06 16:02:55.317,0FCDC0636A8AC008A1559457BA70F8E0,2021-12-06 16:02:55.317,2021-12-06 16:02:55.317,NULL
5,https://www.yelp.com/biz/turkuaz-grill-riverhead?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&hrid=2BgoAXXzaBpVCIdOzdWM-g&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=2FlPnf5brM1FB2UTY6vGnQ,"I highly recommend this place, not only for their tasty food! But the service! Such a hidden gem! If you're ever in the area, worth a try!",5,4,NULL,2021-11-27 16:09:41.000,2021-12-06 16:02:55.317,21E0E7459AD6665F5A178C1708D04F09,2021-12-06 16:02:55.317,2021-12-06 16:02:55.317,NULL
1,https://www.yelp.com/biz/pine-coffee-supply-pinedale?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&hrid=veCFBCoxT2Zg22-1On2nag&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=2FlPnf5brM1FB2UTY6vGnQ,Visited Pinedale on a whim and this place is a little gem! I loved the garage door and murals - it created a very modern ambiance and let in the lovely warm...,5,5,NULL,2021-06-05 16:41:37.000,2021-12-06 15:02:55.317,6221ABA86B084602DF8DE7F2D33263E5,2021-12-06 15:02:55.317,2021-12-06 15:02:55.317,2021-12-06 16:02:55.317
2,https://www.yelp.com/biz/pine-coffee-supply-pinedale?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&hrid=kBEtq46Afm_h1pIrNDMbWQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=2FlPnf5brM1FB2UTY6vGnQ,The worst cup of market value coffee I have ever had. Fifty cents for cream. The girls working the counter were too busy talking to serve customers,1,1,NULL,2021-09-03 13:17:37.000,2021-12-06 15:02:55.317,400D123B610239533F4D5BE6DB626ADA,2021-12-06 15:02:55.317,2021-12-06 15:02:55.317,2021-12-06 16:02:55.317
3,https://www.yelp.com/biz/pine-coffee-supply-pinedale?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&hrid=Q5-dXXyBhHbQOM89grHWlg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=2FlPnf5brM1FB2UTY6vGnQ,Best coffee in town. Oat-milk latte was a creamy espresso bomb tha

In [54]:
-- current source table 
select * FROM dbo.Review

(5 rows affected)

Total execution time: 00:00:00.014

ReviewID,ReviewURL,ReviewExtract,ReviewRating,UserID,BusinessID,CreatedAt,InsertedAt
1,https://www.yelp.com/biz/pine-coffee-supply-pinedale?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&hrid=veCFBCoxT2Zg22-1On2nag&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=2FlPnf5brM1FB2UTY6vGnQ,Visited Pinedale on a whim and this place is a little gem! I loved the garage door and murals - it created a very modern ambiance and let in the lovely warm...,5,5,NULL,2021-06-05 16:41:37.000,2021-12-06 15:02:55.317
2,https://www.yelp.com/biz/pine-coffee-supply-pinedale?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&hrid=kBEtq46Afm_h1pIrNDMbWQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=2FlPnf5brM1FB2UTY6vGnQ,The worst cup of market value coffee I have ever had. Fifty cents for cream. The girls working the counter were too busy talking to serve customers,1,1,NULL,2021-09-03 13:17:37.000,2021-12-06 15:02:55.317
3,https://www.yelp.com/biz/pine-coffee-supply-pinedale?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&hrid=Q5-dXXyBhHbQOM89grHWlg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=2FlPnf5brM1FB2UTY6vGnQ,Best coffee in town. Oat-milk latte was a creamy espresso bomb that blew up my taste buds. Super cool shop and decor.,5,3,NULL,2021-10-14 10:47:04.000,2021-12-06 15:02:55.317
4,https://www.yelp.com/biz/turkuaz-grill-riverhead?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&hrid=Gv29EXhnHu5Xe4bvsktiGw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=2FlPnf5brM1FB2UTY6vGnQ,"I punched in ""ethnic eats"" in my trusty Yelp search bar while having a relaxing weekend in the Hamptons and Turkuaz was the first up on the list. Being a...",5,2,NULL,2021-10-15 18:06:41.000,2021-12-06 15:02:55.317
5,https://www.yelp.com/biz/turkuaz-grill-riverhead?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&hrid=2BgoAXXzaBpVCIdOzdWM-g&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=2FlPnf5brM1FB2UTY6vGnQ,"I highly recommend this place, not only for their tasty food! But the service! Such a hidden gem! If you're ever in the area, worth a try!",5,4,NULL,2021-11-27 16:09:41.000,2021-12-06 15:02:55.317


In [55]:
-- first were going to move our data to another table
select *
into dbo.Review2
from dbo.Review

(5 rows affected)

Total execution time: 00:00:00.005

In [67]:
-- BEGIN TRANSACTION
-- insert into dbo.Review
-- VALUES
-- (7, N'http://getmoney.com', N'idek why I cáme I dont like cajun', 4, 5, 5, DATEFROMPARTS(2021, 11, 12), GETDATE()),
-- (6, N'http://getmoney.com', N'ahora me toca hablar español', 4, 5, 5, DATEFROMPARTS(2021, 11, 12), DATETIMEFROMPARTS(2021, 12, 06, 18,0,0,0))
-- COMMIT TRAN
-- BEGIN TRANSACTION
-- UPdate dbo.Review
-- set BusinessID=4, InsertedAt= DATETIMEFROMPARTS(2021,12,06,18,0,0,0)
-- where ReviewID=5
-- COMMIT TRAN
select * FROM dbo.Review

(7 rows affected)

Total execution time: 00:00:00.006

ReviewID,ReviewURL,ReviewExtract,ReviewRating,UserID,BusinessID,CreatedAt,InsertedAt
1,https://www.yelp.com/biz/pine-coffee-supply-pinedale?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&hrid=veCFBCoxT2Zg22-1On2nag&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=2FlPnf5brM1FB2UTY6vGnQ,Visited Pinedale on a whim and this place is a little gem! I loved the garage door and murals - it created a very modern ambiance and let in the lovely warm...,5,5,NULL,2021-06-05 16:41:37.000,2021-12-06 16:02:55.317
2,https://www.yelp.com/biz/pine-coffee-supply-pinedale?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&hrid=kBEtq46Afm_h1pIrNDMbWQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=2FlPnf5brM1FB2UTY6vGnQ,The worst cup of market value coffee I have ever had. Fifty cents for cream. The girls working the counter were too busy talking to serve customers,1,1,NULL,2021-09-03 13:17:37.000,2021-12-06 16:02:55.317
3,https://www.yelp.com/biz/pine-coffee-supply-pinedale?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&hrid=Q5-dXXyBhHbQOM89grHWlg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=2FlPnf5brM1FB2UTY6vGnQ,Best coffee in town. Oat-milk latte was a creamy espresso bomb that blew up my taste buds. Super cool shop and decor.,5,3,NULL,2021-10-14 10:47:04.000,2021-12-06 16:02:55.317
4,https://www.yelp.com/biz/turkuaz-grill-riverhead?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&hrid=Gv29EXhnHu5Xe4bvsktiGw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=2FlPnf5brM1FB2UTY6vGnQ,"I punched in ""ethnic eats"" in my trusty Yelp search bar while having a relaxing weekend in the Hamptons and Turkuaz was the first up on the list. Being a...",5,2,NULL,2021-10-15 18:06:41.000,2021-12-06 16:02:55.317
5,https://www.yelp.com/biz/turkuaz-grill-riverhead?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&hrid=2BgoAXXzaBpVCIdOzdWM-g&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=2FlPnf5brM1FB2UTY6vGnQ,"I highly recommend this place, not only for their tasty food! But the service! Such a hidden gem! If you're ever in the area, worth a try!",5,4,4,2021-11-27 16:09:41.000,2021-12-06 18:00:00.000
7,http://getmoney.com,idek why I cáme I dont like cajun,4,5,5,2021-11-12 00:00:00.000,2021-12-08 17:21:20.790
6,http://getmoney.com,ahora me toca hablar español,4,5,5,2021-11-12 00:00:00.000,2021-12-06 18:00:00.000


In [ ]:
-- now well run the models'

-- dbt snapshot --select snap_review1
-- dbt run --select DimReview
-- dbt run --select DimBusiness

In [69]:
-- updated reviewid 5 and added 6 and 7
select * from Snapshots.snap_review

(13 rows affected)

Total execution time: 00:00:00.017

ReviewID,ReviewURL,ReviewExtract,ReviewRating,UserID,BusinessID,CreatedAt,InsertedAt,dbt_scd_id,dbt_updated_at,dbt_valid_from,dbt_valid_to
1,https://www.yelp.com/biz/pine-coffee-supply-pinedale?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&hrid=veCFBCoxT2Zg22-1On2nag&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=2FlPnf5brM1FB2UTY6vGnQ,Visited Pinedale on a whim and this place is a little gem! I loved the garage door and murals - it created a very modern ambiance and let in the lovely warm...,5,5,NULL,2021-06-05 16:41:37.000,2021-12-06 16:02:55.317,815880AA9E7400140169211360EC87C9,2021-12-06 16:02:55.317,2021-12-06 16:02:55.317,NULL
2,https://www.yelp.com/biz/pine-coffee-supply-pinedale?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&hrid=kBEtq46Afm_h1pIrNDMbWQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=2FlPnf5brM1FB2UTY6vGnQ,The worst cup of market value coffee I have ever had. Fifty cents for cream. The girls working the counter were too busy talking to serve customers,1,1,NULL,2021-09-03 13:17:37.000,2021-12-06 16:02:55.317,04DB06711F4016E8076E5E3A454B81FF,2021-12-06 16:02:55.317,2021-12-06 16:02:55.317,NULL
3,https://www.yelp.com/biz/pine-coffee-supply-pinedale?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&hrid=Q5-dXXyBhHbQOM89grHWlg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=2FlPnf5brM1FB2UTY6vGnQ,Best coffee in town. Oat-milk latte was a creamy espresso bomb that blew up my taste buds. Super cool shop and decor.,5,3,NULL,2021-10-14 10:47:04.000,2021-12-06 16:02:55.317,CFD45FF4049D52B55B72E755079ACA8E,2021-12-06 16:02:55.317,2021-12-06 16:02:55.317,NULL
4,https://www.yelp.com/biz/turkuaz-grill-riverhead?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&hrid=Gv29EXhnHu5Xe4bvsktiGw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=2FlPnf5brM1FB2UTY6vGnQ,"I punched in ""ethnic eats"" in my trusty Yelp search bar while having a relaxing weekend in the Hamptons and Turkuaz was the first up on the list. Being a...",5,2,NULL,2021-10-15 18:06:41.000,2021-12-06 16:02:55.317,0FCDC0636A8AC008A1559457BA70F8E0,2021-12-06 16:02:55.317,2021-12-06 16:02:55.317,NULL
1,https://www.yelp.com/biz/pine-coffee-supply-pinedale?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&hrid=veCFBCoxT2Zg22-1On2nag&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=2FlPnf5brM1FB2UTY6vGnQ,Visited Pinedale on a whim and this place is a little gem! I loved the garage door and murals - it created a very modern ambiance and let in the lovely warm...,5,5,NULL,2021-06-05 16:41:37.000,2021-12-06 15:02:55.317,6221ABA86B084602DF8DE7F2D33263E5,2021-12-06 15:02:55.317,2021-12-06 15:02:55.317,2021-12-06 16:02:55.317
2,https://www.yelp.com/biz/pine-coffee-supply-pinedale?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&hrid=kBEtq46Afm_h1pIrNDMbWQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=2FlPnf5brM1FB2UTY6vGnQ,The worst cup of market value coffee I have ever had. Fifty cents for cream. The girls working the counter were too busy talking to serve customers,1,1,NULL,2021-09-03 13:17:37.000,2021-12-06 15:02:55.317,400D123B610239533F4D5BE6DB626ADA,2021-12-06 15:02:55.317,2021-12-06 15:02:55.317,2021-12-06 16:02:55.317
3,https://www.yelp.com/biz/pine-coffee-supply-pinedale?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&hrid=Q5-dXXyBhHbQOM89grHWlg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=2FlPnf5brM1FB2UTY6vGnQ,Best coffee in town. Oat-milk latte was a creamy espresso bomb that blew up my taste buds. Super cool shop and decor.,5,3,NULL,2021-10-14 10:47:04.000,2021-12-06 15:02:55.317,27121E463B43EE65D6B2229AB0B4152C,2021-12-06 15:02:55.317,2021-12-06 15:02:55.317,2021-12-06 16:02:55.317
4,https://www.yelp.com/biz/turkuaz-grill-riverhead?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&hrid=Gv29EXhnHu5Xe4bvsktiGw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=2FlPnf5brM1FB2UTY6vGnQ,"I punched in ""ethnic eats"" in my trusty Yelp search bar while havin

finish the joins

In [70]:
-- review dimension
select 
*
from _Production.DimReview

(3 rows affected)

Total execution time: 00:00:00.017

ReviewKey,ReviewSourceKey,ReviewURL,ReviewExtract,ReviewRating,CreatedAt,InsertedAt,UserKey,BusinessKey,ValidFrom,ValidTo,is_current
1,5,https://www.yelp.com/biz/turkuaz-grill-riverhead?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&hrid=2BgoAXXzaBpVCIdOzdWM-g&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=2FlPnf5brM1FB2UTY6vGnQ,"I highly recommend this place, not only for their tasty food! But the service! Such a hidden gem! If you're ever in the area, worth a try!",5,2021-11-27 16:09:41.000,2021-12-06 18:00:00.000,4,4,2021-12-06 18:00:00.000,9999-12-31 00:00:00.000,1
2,6,http://getmoney.com,ahora me toca hablar español,4,2021-11-12 00:00:00.000,2021-12-06 18:00:00.000,6,5,2021-12-06 18:00:00.000,9999-12-31 00:00:00.000,1
3,7,http://getmoney.com,idek why I cáme I dont like cajun,4,2021-11-12 00:00:00.000,2021-12-08 17:21:20.790,5,6,2021-12-08 17:21:20.790,9999-12-31 00:00:00.000,1


From this query we can decide to group on BusinessSourceKey and from there proceed to the all the reviews

in the system for a business. There would be no issue with duplicate as the work has already been done

to make sure a review is only paired with one "instance" of a business and user.

However easily one can go back to the sourcekeys for a business or user and use that for aggregation

In [ ]:
select 
dr.ReviewKey,
dr.ReviewSourceKey,
dr.userkey,
dr.businesskey,
db.BusinessSourceKey,
du.UserSourceKey,
dr.ReviewExtract,
db.BusinessName,
dr.CreatedAt,
dr.InsertedAt,
db.ValidFrom bus_valid_to,
db.ValidTo bus_valid_from,
du.ValidFrom user_valid_to,
du.ValidTo user_valid_from
from _Production.DimReview dr
INNER JOIN _Production.DimBusiness db on dr.businesskey=db.BusinessKey
INNER JOIN _Production.DimUser du on dr.UserKey=du.UserKey

DimEvent

In [72]:
-- current event source table status
select * FROM dbo.Event

(5 rows affected)

Total execution time: 00:00:00.024

EventID,BusinessID,EventName,Attending,CostOfAttending,is_free,EventDescription,Interested,CityID,latitude,longitude,ZipCode,StartTime,EndTime,TicketsUrl,EventSiteUrl,CancelDate,OfficialDate,CreatedAt,LastEditedWhen
1,NULL,Business meeting,1,0,True,To all business minded people you can't miss this opportunity. We have a special price for a limited time to join. Come all and change the trajectory of not...,0,NULL,32,-83,31206,2018-01-11 19:00:00.0000000 -05:00,NULL,NULL,https://www.yelp.com/events/macon-business-meeting?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_event_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,NULL,NULL,2021-11-28,2021-12-06 15:02:56.457
2,NULL,Enjoy an Apple Turnover at Krystal,1,0,False,"As sweet (and American) as apple pie, our apple turnover is filled with apples, mixed with the perfect blend of spices and then stuffed into a delicious,...",0,NULL,32,-83,31206,2021-07-05 00:00:00.0000000 -04:00,NULL,NULL,https://www.yelp.com/events/macon-enjoy-an-apple-turnover-at-krystal-2?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_event_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,NULL,NULL,2021-11-28,2021-12-06 15:02:56.457
3,NULL,Enjoy an Apple Turnover at Krystal,1,0,False,"As sweet (and American) as apple pie, our apple turnover is filled with apples, mixed with the perfect blend of spices and then stuffed into a delicious,...",0,NULL,32,-83,31206,2021-07-05 00:00:00.0000000 -04:00,NULL,NULL,https://www.yelp.com/events/macon-enjoy-an-apple-turnover-at-krystal-4?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_event_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,NULL,NULL,2021-11-28,2021-12-06 15:02:56.457
4,NULL,krystal has you covered for your morning and late-night cravings,1,0,False,"ATLANTA, GA -- When your cravings hit, Krystal Restaurants LLC has you covered, whether it's early in the morning or late at night. For a limited time,...",0,NULL,32,-83,31206,2021-06-30 11:00:00.0000000 -04:00,NULL,NULL,https://www.yelp.com/events/macon-krystal-has-you-covered-for-your-morning-and-late-night-cravings-2?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_event_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,NULL,NULL,2021-11-28,2021-12-06 15:02:56.457
5,5,krystal has you covered for your morning and late-night cravings,1,0,False,"ATLANTA, GA -- When your cravings hit, Krystal Restaurants LLC has you covered, whether it's early in the morning or late at night. For a limited time,...",0,NULL,32,-83,31206,2021-06-30 11:00:00.0000000 -04:00,NULL,NULL,https://www.yelp.com/events/macon-krystal-has-you-covered-for-your-morning-and-late-night-cravings-4?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_event_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,NULL,NULL,2021-12-07,2021-12-07 01:00:00.000


In [73]:

select * from _Production.DimEvent

(1 row affected)

Total execution time: 00:00:00.023

EventSourceKey,BusinessSourceKey,EventName,Attending,CostOfAttending,is_free,EventDescription,Interested,CitySourceKey,latitude,longitude,ZipCode,StartTime,EndTime,TicketsUrl,EventSiteUrl,CancelDate,OfficialDate,CreatedAt,LastEditedWhen
5,5,krystal has you covered for your morning and late-night cravings,1,0,False,"ATLANTA, GA -- When your cravings hit, Krystal Restaurants LLC has you covered, whether it's early in the morning or late at night. For a limited time,...",0,NULL,32,-83,31206,2021-06-30 11:00:00.0000000 -04:00,NULL,NULL,https://www.yelp.com/events/macon-krystal-has-you-covered-for-your-morning-and-late-night-cravings-4?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_event_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,NULL,NULL,2021-12-07,2021-12-07 01:00:00.000


Now we're going to introduce a new event and an updated event to show an accumlating snapshot table

In [ ]:
-- inadvertently create duplicates
insert into dbo.Event
select top(2) *
from dbo.Event2
order by EventID

begin TRANSACTION
UPDATE dbo.Event 
set EventID=5, BusinessID=5, CreatedAt= DATEFROMPARTS(2021,12,7), LastEditedWhen=GETDATE(), CancelDate= cast(GETDATE() as date)
where EventID = 1

UPDATE dbo.Event 
set EventID=6, BusinessID=1, CreatedAt=DATETIMEFROMPARTS(2021,12,07,0,0,0,0)  , LastEditedWhen=GETDATE()
where EventID =2
COMMIT TRAN
ROLLBACK TRAN

-- to get rid of duplicates
select distinct *
into dbo.Event3
from dbo.event

BEGIN TRAN
TRUNCATE table dbo.event
insert into dbo.Event
SELECT distinct *
from dbo.Event3
COMMIT TRAN

delete from dbo.Event
where BusinessID=5 and CancelDate is null

In [74]:
-- our new source table
select * from dbo.Event

(4 rows affected)

Total execution time: 00:00:00.020

EventID,BusinessID,EventName,Attending,CostOfAttending,is_free,EventDescription,Interested,CityID,latitude,longitude,ZipCode,StartTime,EndTime,TicketsUrl,EventSiteUrl,CancelDate,OfficialDate,CreatedAt,LastEditedWhen
3,NULL,Enjoy an Apple Turnover at Krystal,1,0,False,"As sweet (and American) as apple pie, our apple turnover is filled with apples, mixed with the perfect blend of spices and then stuffed into a delicious,...",0,NULL,32,-83,31206,2021-07-05 00:00:00.0000000 -04:00,NULL,NULL,https://www.yelp.com/events/macon-enjoy-an-apple-turnover-at-krystal-4?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_event_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,NULL,NULL,2021-11-28,2021-12-06 15:02:56.457
4,NULL,krystal has you covered for your morning and late-night cravings,1,0,False,"ATLANTA, GA -- When your cravings hit, Krystal Restaurants LLC has you covered, whether it's early in the morning or late at night. For a limited time,...",0,NULL,32,-83,31206,2021-06-30 11:00:00.0000000 -04:00,NULL,NULL,https://www.yelp.com/events/macon-krystal-has-you-covered-for-your-morning-and-late-night-cravings-2?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_event_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,NULL,NULL,2021-11-28,2021-12-06 15:02:56.457
5,5,Business meeting,1,0,True,To all business minded people you can't miss this opportunity. We have a special price for a limited time to join. Come all and change the trajectory of not...,0,NULL,32,-83,31206,2018-01-11 19:00:00.0000000 -05:00,NULL,NULL,https://www.yelp.com/events/macon-business-meeting?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_event_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,2021-12-08,NULL,2021-12-07,2021-12-08 19:25:16.830
6,1,Enjoy an Apple Turnover at Krystal,1,0,False,"As sweet (and American) as apple pie, our apple turnover is filled with apples, mixed with the perfect blend of spices and then stuffed into a delicious,...",0,NULL,32,-83,31206,2021-07-05 00:00:00.0000000 -04:00,NULL,NULL,https://www.yelp.com/events/macon-enjoy-an-apple-turnover-at-krystal-2?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_event_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,NULL,NULL,2021-12-07,2021-12-08 19:25:16.830


Now we can see that our event was updated in the system with the new cancel date of today, also a new event was able to be added as well

In [75]:
select * from _Production.DimEvent

(2 rows affected)

Total execution time: 00:00:00.015

EventSourceKey,BusinessSourceKey,EventName,Attending,CostOfAttending,is_free,EventDescription,Interested,CitySourceKey,latitude,longitude,ZipCode,StartTime,EndTime,TicketsUrl,EventSiteUrl,CancelDate,OfficialDate,CreatedAt,LastEditedWhen
6,1,Enjoy an Apple Turnover at Krystal,1,0,False,"As sweet (and American) as apple pie, our apple turnover is filled with apples, mixed with the perfect blend of spices and then stuffed into a delicious,...",0,NULL,32,-83,31206,2021-07-05 00:00:00.0000000 -04:00,NULL,NULL,https://www.yelp.com/events/macon-enjoy-an-apple-turnover-at-krystal-2?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_event_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,NULL,NULL,2021-12-07,2021-12-08 19:25:16.830
5,5,Business meeting,1,0,True,To all business minded people you can't miss this opportunity. We have a special price for a limited time to join. Come all and change the trajectory of not...,0,NULL,32,-83,31206,2018-01-11 19:00:00.0000000 -05:00,NULL,NULL,https://www.yelp.com/events/macon-business-meeting?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_event_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,2021-12-08,NULL,2021-12-07,2021-12-08 19:25:16.830


Here we will update zoos and see what a category change looks like

In [1]:
select * from dbo.businesscategory WHERE CategoryID = 592


(1 row affected)

Total execution time: 00:00:00.012

CategoryID,CategoryName,LastEditedWhen
592,Zoos,2021-12-06 15:02:53.597


In [2]:
BEGIN TRAN
update dbo.businesscategory
SET CategoryName = N'El zoológico', LastEditedWhen= DATETIMEFROMPARTS(2021,12,07,11,0,0,0)
WHERE CategoryID = 592
COMMIT TRANSACTION
select * 
FROM businesscategory
where CategoryID = 592


(1 row affected)

(1 row affected)

Total execution time: 00:00:00.014

CategoryID,CategoryName,LastEditedWhen
592,El zoológico,2021-12-07 11:00:00.000


In [ ]:
-- now we can run
-- dbt snapshot --select snap_dimbusinesscategory 
-- dbt run --select DimBusinessCategory
-- dbt run --select DimBusinessCategoryBridge

In [3]:
-- now we can see the previous category name zoo is expired and the new El zoológico 
-- is made effective
select * from Snapshots.snap_dimbusinesscategory ORDER BY CategoryID DESC

(593 rows affected)

Total execution time: 00:00:00.047

CategoryID,CategoryName,LastEditedWhen,dbt_scd_id,dbt_updated_at,dbt_valid_from,dbt_valid_to
592,Zoos,2021-12-06 15:02:53.597,F2005284D4A306B7DEF8D198D292DA64,2021-12-06 15:02:53.597,2021-12-06 15:02:53.597,2021-12-07 11:00:00.000
592,El zoológico,2021-12-07 11:00:00.000,4BFB7ED4CF0EE7931949A4EE31F27681,2021-12-07 11:00:00.000,2021-12-07 11:00:00.000,NULL
591,Ziplining,2021-12-06 15:02:53.597,AFC8B9EF03928CB57D25065E13C5F80D,2021-12-06 15:02:53.597,2021-12-06 15:02:53.597,NULL
590,Yoga,2021-12-06 15:02:53.597,0A49E18C5427BD1D2F5F72A8F1552EC7,2021-12-06 15:02:53.597,2021-12-06 15:02:53.597,NULL
589,Yelp Events,2021-12-06 15:02:53.597,AD80833CDFB60265390BFBA84CC82B3D,2021-12-06 15:02:53.597,2021-12-06 15:02:53.597,NULL
588,Wraps,2021-12-06 15:02:53.597,E8A85D1AB60E2088D8A6C4FEA5ABC392,2021-12-06 15:02:53.597,2021-12-06 15:02:53.597,NULL
587,Women's Clothing,2021-12-06 15:02:53.597,DE6747B734F7A7F2EE3C9DFAE33A5E2E,2021-12-06 15:02:53.597,2021-12-06 15:02:53.597,NULL
586,Wineries,2021-12-06 15:02:53.597,A2EF69A129C3D43FFC75A6D747E1F8A8,2021-12-06 15:02:53.597,2021-12-06 15:02:53.597,NULL
585,Wine Tours,2021-12-06 15:02:53.597,042EEC9CA6D93C20EF82A0714421B0EC,2021-12-06 15:02:53.597,2021-12-06 15:02:53.597,NULL
584,Wine Tasting Room,2021-12-06 15:02:53.597,5F744B878A2E6423C73E67B7B35FBC9C,2021-12-06 15:02:53.597,2021-12-06 15:02:53.597,NULL


In [6]:
-- due to not having created the snapshot yet with "ORDER BY CategoryID ASC, LastEditedWhen ASC"
-- the new name gets a odd placement in the businesscategorykey
SELECT * 
from _Production.DimBusinessCategory 
ORDER by CategorySourceKey DESC

(593 rows affected)

Total execution time: 00:00:00.179

BusinessCategoryKey,CategorySourceKey,BusinessCategoryName,ValidFrom,ValidTo,is_current
593,592,Zoos,2021-12-06 15:02:53.597,2021-12-07 11:00:00.000,0
186,592,El zoológico,2021-12-07 11:00:00.000,9999-12-31 00:00:00.000,1
592,591,Ziplining,2021-12-06 15:02:53.597,9999-12-31 00:00:00.000,1
591,590,Yoga,2021-12-06 15:02:53.597,9999-12-31 00:00:00.000,1
590,589,Yelp Events,2021-12-06 15:02:53.597,9999-12-31 00:00:00.000,1
589,588,Wraps,2021-12-06 15:02:53.597,9999-12-31 00:00:00.000,1
588,587,Women's Clothing,2021-12-06 15:02:53.597,9999-12-31 00:00:00.000,1
587,586,Wineries,2021-12-06 15:02:53.597,9999-12-31 00:00:00.000,1
586,585,Wine Tours,2021-12-06 15:02:53.597,9999-12-31 00:00:00.000,1
585,584,Wine Tasting Room,2021-12-06 15:02:53.597,9999-12-31 00:00:00.000,1


Now for obvious reasons nothing changes here because no business contained at the moment has the same category as the zoo

In [11]:
select * from _production.DimBusinessCategoryBridge

(12 rows affected)

Total execution time: 00:00:00.016

BusinessKey,BusinessCategoryKey,BusinessID,CategoryID,ValidFrom,ValidTo,is_current
1,14,1,17,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,1
1,49,1,52,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,1
2,83,2,84,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,1
2,319,2,319,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,1
4,492,4,492,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,1
4,495,4,495,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,1
1,560,1,559,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,1
3,565,3,564,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,1
2,164,2,165,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,1
5,98,5,99,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,1


In [10]:
select 
*
from _production.DimBusinessCategoryBridge bcb
INNER JOIN _Production.DimBusiness db on bcb.businesskey = db.businesskey
INNER JOIN _Production.DimBusinessCategory dbc on bcb.BusinessCategoryKey=dbc.BusinessCategoryKey

(12 rows affected)

Total execution time: 00:00:00.066

BusinessKey,BusinessCategoryKey,BusinessID,CategoryID,ValidFrom,ValidTo,is_current,BusinessKey,BusinessSourceKey,BusinessName,ChainName,CitySourceKey,PaymentLevelSymbol,PaymentLevelName,Longitude,Latitude,AddressLine1,AddressLine2,AddressLine3,ZipCode,BusinessPhone,BusinessURL,is_closed,DistanceToCounty,ValidFrom,ValidTo,is_current,BusinessCategoryKey,CategorySourceKey,BusinessCategoryName,ValidFrom,ValidTo,is_current
1,560,1,559,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,1,1,1,central-montgomery-3,Central,1,$$$,High,-86.310560,32.380180,129 Coosa St,NULL,NULL,36104,13345171155,https://www.yelp.com/biz/central-montgomery-3?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,36172,2021-12-06 15:02:55.797,9999-12-31 00:00:00.000,1,560,559,Venues & Event Spaces,2021-12-06 15:02:53.597,9999-12-31 00:00:00.000,1
2,164,2,165,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,1,2,2,d-road-cafe-montgomery,D' Road Cafe,1,$$,Low,-86.309962,32.376970,121 Montgomery St,NULL,NULL,36104,13343282938,https://www.yelp.com/biz/d-road-cafe-montgomery?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,36365,2021-12-06 15:02:55.797,9999-12-31 00:00:00.000,1,164,165,Desserts,2021-12-06 15:02:53.597,9999-12-31 00:00:00.000,1
3,565,3,564,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,1,3,3,lÃ¡taste-prattville-2,LÃ¡Taste,2,$$,Low,-86.420700,32.459576,1868 E Main St,NULL,NULL,36066,13343561149,https://www.yelp.com/biz/l%C3%A1taste-prattville-2?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,23567,2021-12-06 15:02:55.797,9999-12-31 00:00:00.000,1,565,564,Vietnamese,2021-12-06 15:02:53.597,9999-12-31 00:00:00.000,1
4,495,4,495,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,1,4,4,mrs-b-s-home-cooking-montgomery,Mrs Bâ€™s Home Cooking,1,$$,Low,-86.348587,32.366208,415 Airbase Blvd,NULL,NULL,36104,13342645495,https://www.yelp.com/biz/mrs-b-s-home-cooking-montgomery?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,33610,2021-12-06 15:02:55.797,9999-12-31 00:00:00.000,1,495,495,Southern,2021-12-06 15:02:53.597,9999-12-31 00:00:00.000,1
5,166,5,167,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,1,5,5,uncle-micks-cajun-market-and-cafe-prattville,Uncle Mick's Cajun Market & Cafe,2,$$,Low,-86.474090,32.460040,136 W Main St,NULL,NULL,36067,13343611020,https://www.yelp.com/biz/uncle-micks-cajun-market-and-cafe-prattville?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,18704,2021-12-06 15:02:55.797,2021-12-08 15:00:52.453,0,166,167,Diners,2021-12-06 15:02:53.597,9999-12-31 00:00:00.000,1
1,49,1,52,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,1,1,1,central-montgomery-3,Central,1,$$$,High,-86.310560,32.380180,129 Coosa St,NULL,NULL,36104,13345171155,https://www.yelp.com/biz/central-montgomery-3?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,36172,2021-12-06 15:02:55.797,9999-12-31 00:00:00.000,1,49,52,Bars,2021-12-06 15:02:53.597,9999-12-31 00:00:00.000,1
1,14,1,17,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,1,1,1,central-montgomery-3,Central,1,$$$,High,-86.310560,32.380180,129 Coosa St,NULL,NULL,36104,13345171155,https://www.yelp.com/biz/central-montgomery-3?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,36172,2021-12-06 15:02:55.797,9999-12-31 00:00:00.000,1,14,17,American (Traditional),2021-12-06 15:02:53.597,9999-12-31 00:00:00.000,1
2,319,2,319,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,1,2,2,d-road-cafe-montgomery,D' Road Cafe,1,$$,Low,-86.309962,32.376970,121 

Now we're going to modify an entry in the business category bridge source table so that it includes the zoo categoryname

In [12]:
INSERT dbo.BusinessCategoryBridge (BusinessID, CategoryID, LastEditedWhen)
VALUES
(5,592, DATETIMEFROMPARTS(2021,12,7,10,59,52,203))
select * from dbo.BusinessCategoryBridge

(1 row affected)

(13 rows affected)

Total execution time: 00:00:00.137

BusinessID,CategoryID,LastEditedWhen,SnapshotCompKey
1,17,2021-12-07 10:59:52.203,1-17
1,52,2021-12-07 10:59:52.203,1-52
2,84,2021-12-07 10:59:52.203,2-84
5,99,2021-12-07 10:59:52.203,5-99
2,165,2021-12-07 10:59:52.203,2-165
5,167,2021-12-07 10:59:52.203,5-167
2,319,2021-12-07 10:59:52.203,2-319
4,492,2021-12-07 10:59:52.203,4-492
4,495,2021-12-07 10:59:52.203,4-495
1,559,2021-12-07 10:59:52.203,1-559


In [ ]:
-- now we can run
-- dbt snapshot --select snap_dimbusinesscategorybridge 
-- dbt run --select DimBusinessCategoryBridge

In [13]:
-- now a new connection between the business and the zoo
select * from Snapshots.snap_businesscategorybridge

(13 rows affected)

Total execution time: 00:00:00.020

BusinessID,CategoryID,LastEditedWhen,SnapshotCompKey,dbt_scd_id,dbt_updated_at,dbt_valid_from,dbt_valid_to
1,17,2021-12-07 10:59:52.203,1-17,A7A8C905D0B33D894F6D0C853F0BA304,2021-12-07 10:59:52.203,2021-12-07 10:59:52.203,NULL
1,52,2021-12-07 10:59:52.203,1-52,D0745AF512F4856C71AB722F42E51DB9,2021-12-07 10:59:52.203,2021-12-07 10:59:52.203,NULL
2,84,2021-12-07 10:59:52.203,2-84,F083E15F7DAF8ECB17774B68304C121D,2021-12-07 10:59:52.203,2021-12-07 10:59:52.203,NULL
5,99,2021-12-07 10:59:52.203,5-99,A7AA206D7C49992BBC0A433E2BB5BDF8,2021-12-07 10:59:52.203,2021-12-07 10:59:52.203,NULL
2,165,2021-12-07 10:59:52.203,2-165,2C52C9D718EDD5BAA6927E476337734F,2021-12-07 10:59:52.203,2021-12-07 10:59:52.203,NULL
5,167,2021-12-07 10:59:52.203,5-167,971631B6506C5CE25B0A1A55D04A9899,2021-12-07 10:59:52.203,2021-12-07 10:59:52.203,NULL
2,319,2021-12-07 10:59:52.203,2-319,52A73CE2633B77734B7CF43E9440F446,2021-12-07 10:59:52.203,2021-12-07 10:59:52.203,NULL
4,492,2021-12-07 10:59:52.203,4-492,00B10FFAFDF21B5032CF43013D8AF9FE,2021-12-07 10:59:52.203,2021-12-07 10:59:52.203,NULL
4,495,2021-12-07 10:59:52.203,4-495,4CEDBA1145491D053A9C965A6CCBCA8E,2021-12-07 10:59:52.203,2021-12-07 10:59:52.203,NULL
1,559,2021-12-07 10:59:52.203,1-559,EE61A01B4A20C490C1CC7D36D3735DAC,2021-12-07 10:59:52.203,2021-12-07 10:59:52.203,NULL


Here we can see that witth our search argument logic implemented the "zoo" and "el zoologico" category both appear for the first instance of business #5 however since "el zoologico" was the only category instance available during the second instance of business #5 we see that alone reflected in the results

The other categories also carry on to this new instance since they are still valid

In [50]:
select * from _production.DimBusinessCategoryBridge

(18 rows affected)

Total execution time: 00:00:00.013

BusinessKey,BusinessCategoryKey,BusinessID,CategoryID,ValidFrom,ValidTo,is_current
1,14,1,17,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,0
1,49,1,52,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,0
2,83,2,84,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,0
6,98,5,99,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,0
6,164,5,165,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,0
6,166,5,167,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,0
2,319,2,319,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,0
4,492,4,492,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,0
4,495,4,495,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,0
1,560,1,559,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,0


Now we'll add a new business instance after making a deletion from the business category bridge

I'll delete on business id 4 and business category id 492

In [52]:
DELETE FROM dbo.BusinessCategoryBridge
WHERE BusinessID=4 AND CategoryID=492
select * from dbo.BusinessCategoryBridge

(0 rows affected)

(12 rows affected)

Total execution time: 00:00:00.023

BusinessID,CategoryID,LastEditedWhen,SnapshotCompKey
1,17,2021-12-07 10:59:52.203,1-17
1,52,2021-12-07 10:59:52.203,1-52
2,84,2021-12-07 10:59:52.203,2-84
5,99,2021-12-07 10:59:52.203,5-99
2,165,2021-12-07 10:59:52.203,2-165
5,167,2021-12-07 10:59:52.203,5-167
2,319,2021-12-07 10:59:52.203,2-319
4,495,2021-12-07 10:59:52.203,4-495
1,559,2021-12-07 10:59:52.203,1-559
3,564,2021-12-07 10:59:52.203,3-564


Now we can run our snapshot model

dbt snapshot --select snap\_dimbusinesscategorybridge  

 and verify the invalidation

In [54]:
SELECT * from Snapshots.snap_businesscategorybridge

(13 rows affected)

Total execution time: 00:00:00.017

BusinessID,CategoryID,LastEditedWhen,SnapshotCompKey,dbt_scd_id,dbt_updated_at,dbt_valid_from,dbt_valid_to
1,17,2021-12-07 10:59:52.203,1-17,A7A8C905D0B33D894F6D0C853F0BA304,2021-12-07 10:59:52.203,2021-12-07 10:59:52.203,NULL
1,52,2021-12-07 10:59:52.203,1-52,D0745AF512F4856C71AB722F42E51DB9,2021-12-07 10:59:52.203,2021-12-07 10:59:52.203,NULL
2,84,2021-12-07 10:59:52.203,2-84,F083E15F7DAF8ECB17774B68304C121D,2021-12-07 10:59:52.203,2021-12-07 10:59:52.203,NULL
5,99,2021-12-07 10:59:52.203,5-99,A7AA206D7C49992BBC0A433E2BB5BDF8,2021-12-07 10:59:52.203,2021-12-07 10:59:52.203,NULL
2,165,2021-12-07 10:59:52.203,2-165,2C52C9D718EDD5BAA6927E476337734F,2021-12-07 10:59:52.203,2021-12-07 10:59:52.203,NULL
5,167,2021-12-07 10:59:52.203,5-167,971631B6506C5CE25B0A1A55D04A9899,2021-12-07 10:59:52.203,2021-12-07 10:59:52.203,NULL
2,319,2021-12-07 10:59:52.203,2-319,52A73CE2633B77734B7CF43E9440F446,2021-12-07 10:59:52.203,2021-12-07 10:59:52.203,NULL
4,495,2021-12-07 10:59:52.203,4-495,4CEDBA1145491D053A9C965A6CCBCA8E,2021-12-07 10:59:52.203,2021-12-07 10:59:52.203,NULL
1,559,2021-12-07 10:59:52.203,1-559,EE61A01B4A20C490C1CC7D36D3735DAC,2021-12-07 10:59:52.203,2021-12-07 10:59:52.203,NULL
3,564,2021-12-07 10:59:52.203,3-564,11942BD5AD3AFB7FA8DAD0925568FAEE,2021-12-07 10:59:52.203,2021-12-07 10:59:52.203,NULL


Now we'll create a new instance of this business

I have to offset by 2 hours based pn our our final brdige models joins different categories

In [59]:
UPDATE dbo.business
set ChainName = N'Weeney Hut Junior2',LastEditedWhen =  DATEADD(HOUR, 2, GETDATE())
WHERE BusinessID = 4
select * from dbo.business

(1 row affected)

(5 rows affected)

Total execution time: 00:00:00.033

BusinessID,BusinessName,ChainName,AddressLine1,AddressLine2,AddressLine3,Latitude,Longitude,ZipCode,BusinessPhone,BusinessURL,is_closed,DistanceToCounty,CityID,CountyID,StateID,PaymentLevelID,LastEditedWhen
2,d-road-cafe-montgomery,D' Road Cafe,121 Montgomery St,NULL,NULL,32.376970,-86.309962,36104,13343282938,https://www.yelp.com/biz/d-road-cafe-montgomery?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,36365,1,3102,1,1,2021-12-06 15:02:55.797
1,central-montgomery-3,Central,129 Coosa St,NULL,NULL,32.380180,-86.310560,36104,13345171155,https://www.yelp.com/biz/central-montgomery-3?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,36172,1,3102,1,2,2021-12-06 15:02:55.797
5,uncle-micks-cajun-market-and-cafe-prattville,Uncle Mick's Cajun Market & Cafe,136 W Main St,NULL,NULL,32.460040,-86.474090,36067,13343611020,https://www.yelp.com/biz/uncle-micks-cajun-market-and-cafe-prattville?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,1,18704,2,3102,1,1,2021-12-08 15:00:52.453
4,mrs-b-s-home-cooking-montgomery,Weeney Hut Junior2,415 Airbase Blvd,NULL,NULL,32.366208,-86.348587,36104,13342645495,https://www.yelp.com/biz/mrs-b-s-home-cooking-montgomery?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,33610,1,3102,1,1,2021-12-09 23:15:58.777
3,lÃ¡taste-prattville-2,LÃ¡Taste,1868 E Main St,NULL,NULL,32.459576,-86.420700,36066,13343561149,https://www.yelp.com/biz/l%C3%A1taste-prattville-2?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,23567,2,3102,1,1,2021-12-06 15:02:55.797


Now we'll run our snapshot business model to confirm the changes

dbt snapshot --select snap\_dimbusiness1

In [60]:
select * from Snapshots.snap_business

(8 rows affected)

Total execution time: 00:00:00.078

BusinessID,BusinessName,ChainName,AddressLine1,AddressLine2,AddressLine3,Latitude,Longitude,ZipCode,BusinessPhone,BusinessURL,is_closed,DistanceToCounty,CityID,CountyID,StateID,PaymentLevelID,LastEditedWhen,dbt_scd_id,dbt_updated_at,dbt_valid_from,dbt_valid_to
2,d-road-cafe-montgomery,D' Road Cafe,121 Montgomery St,NULL,NULL,32.376970,-86.309962,36104,13343282938,https://www.yelp.com/biz/d-road-cafe-montgomery?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,36365,1,3102,1,1,2021-12-06 15:02:55.797,400D123B610239533F4D5BE6DB626ADA,2021-12-06 15:02:55.797,2021-12-06 15:02:55.797,NULL
1,central-montgomery-3,Central,129 Coosa St,NULL,NULL,32.380180,-86.310560,36104,13345171155,https://www.yelp.com/biz/central-montgomery-3?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,36172,1,3102,1,2,2021-12-06 15:02:55.797,6221ABA86B084602DF8DE7F2D33263E5,2021-12-06 15:02:55.797,2021-12-06 15:02:55.797,NULL
3,lÃ¡taste-prattville-2,LÃ¡Taste,1868 E Main St,NULL,NULL,32.459576,-86.420700,36066,13343561149,https://www.yelp.com/biz/l%C3%A1taste-prattville-2?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,23567,2,3102,1,1,2021-12-06 15:02:55.797,27121E463B43EE65D6B2229AB0B4152C,2021-12-06 15:02:55.797,2021-12-06 15:02:55.797,NULL
5,uncle-micks-cajun-market-and-cafe-prattville,Uncle Mick's Cajun Market & Cafe,136 W Main St,NULL,NULL,32.460040,-86.474090,36067,13343611020,https://www.yelp.com/biz/uncle-micks-cajun-market-and-cafe-prattville?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,1,18704,2,3102,1,1,2021-12-08 15:00:52.453,9454166543A73910DC0ABF2E047E62CC,2021-12-08 15:00:52.453,2021-12-08 15:00:52.453,NULL
5,uncle-micks-cajun-market-and-cafe-prattville,Uncle Mick's Cajun Market & Cafe,136 W Main St,NULL,NULL,32.460040,-86.474090,36067,13343611020,https://www.yelp.com/biz/uncle-micks-cajun-market-and-cafe-prattville?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,18704,2,3102,1,1,2021-12-06 15:02:55.797,1F23392E436CA417A8437A8E74D6CF6C,2021-12-06 15:02:55.797,2021-12-06 15:02:55.797,2021-12-08 15:00:52.453
4,mrs-b-s-home-cooking-montgomery,Mrs Bâ€™s Home Cooking,415 Airbase Blvd,NULL,NULL,32.366208,-86.348587,36104,13342645495,https://www.yelp.com/biz/mrs-b-s-home-cooking-montgomery?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,33610,1,3102,1,1,2021-12-06 15:02:55.797,BE3D6925C6017215C1F1604B58C4A837,2021-12-06 15:02:55.797,2021-12-06 15:02:55.797,2021-12-09 20:53:54.800
4,mrs-b-s-home-cooking-montgomery,Weeney Hut Junior2,415 Airbase Blvd,NULL,NULL,32.366208,-86.348587,36104,13342645495,https://www.yelp.com/biz/mrs-b-s-home-cooking-montgomery?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,33610,1,3102,1,1,2021-12-09 23:15:58.777,C2CBFECBC35E9E4BC031B79FF538297E,2021-12-09 23:15:58.777,2021-12-09 23:15:58.777,NULL
4,mrs-b-s-home-cooking-montgomery,Weeney Hut Junior,415 Airbase Blvd,NULL,NULL,32.366208,-86.348587,36104,13342645495,https://www.yelp.com/biz/mrs-b-s-home-cooking-montgomery?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,33610,1,3102,1,1,2021-12-09 20:53:54.800,CD615E88332B04CA53E4EA0B4A7C85AF,2021-12-09 20:53:54.800,2021-12-09 20:53:54.800,2021-12-09 23:15:58.777


Now we'll rerun the rest of the models all the way to the <span style="color: rgb(174, 129, 255); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">DimBusinessCategoryBridge table</span>

<span style="font-size: 12px; white-space: pre;">dbt run --select dimbusiness1+</span>  

<span style="font-size: 12px; white-space: pre;">dbt run --select dimbcb+</span>

In [1]:
-- our new business instance
select * from _production.DimBusiness

(8 rows affected)

Total execution time: 00:00:00.149

BusinessKey,BusinessSourceKey,BusinessName,ChainName,CitySourceKey,PaymentLevelSymbol,PaymentLevelName,Longitude,Latitude,AddressLine1,AddressLine2,AddressLine3,ZipCode,BusinessPhone,BusinessURL,is_closed,DistanceToCounty,ValidFrom,ValidTo,is_current
1,1,central-montgomery-3,Central,1,$$$,High,-86.310560,32.380180,129 Coosa St,NULL,NULL,36104,13345171155,https://www.yelp.com/biz/central-montgomery-3?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,36172,2021-12-06 15:02:55.797,9999-12-31 00:00:00.000,1
2,2,d-road-cafe-montgomery,D' Road Cafe,1,$$,Low,-86.309962,32.376970,121 Montgomery St,NULL,NULL,36104,13343282938,https://www.yelp.com/biz/d-road-cafe-montgomery?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,36365,2021-12-06 15:02:55.797,9999-12-31 00:00:00.000,1
3,3,lÃ¡taste-prattville-2,LÃ¡Taste,2,$$,Low,-86.420700,32.459576,1868 E Main St,NULL,NULL,36066,13343561149,https://www.yelp.com/biz/l%C3%A1taste-prattville-2?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,23567,2021-12-06 15:02:55.797,9999-12-31 00:00:00.000,1
4,4,mrs-b-s-home-cooking-montgomery,Mrs Bâ€™s Home Cooking,1,$$,Low,-86.348587,32.366208,415 Airbase Blvd,NULL,NULL,36104,13342645495,https://www.yelp.com/biz/mrs-b-s-home-cooking-montgomery?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,33610,2021-12-06 15:02:55.797,2021-12-09 20:53:54.800,0
5,4,mrs-b-s-home-cooking-montgomery,Weeney Hut Junior,1,$$,Low,-86.348587,32.366208,415 Airbase Blvd,NULL,NULL,36104,13342645495,https://www.yelp.com/biz/mrs-b-s-home-cooking-montgomery?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,33610,2021-12-09 20:53:54.800,2021-12-09 23:15:58.777,0
6,4,mrs-b-s-home-cooking-montgomery,Weeney Hut Junior2,1,$$,Low,-86.348587,32.366208,415 Airbase Blvd,NULL,NULL,36104,13342645495,https://www.yelp.com/biz/mrs-b-s-home-cooking-montgomery?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,33610,2021-12-09 23:15:58.777,9999-12-31 00:00:00.000,1
7,5,uncle-micks-cajun-market-and-cafe-prattville,Uncle Mick's Cajun Market & Cafe,2,$$,Low,-86.474090,32.460040,136 W Main St,NULL,NULL,36067,13343611020,https://www.yelp.com/biz/uncle-micks-cajun-market-and-cafe-prattville?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,0,18704,2021-12-06 15:02:55.797,2021-12-08 15:00:52.453,0
8,5,uncle-micks-cajun-market-and-cafe-prattville,Uncle Mick's Cajun Market & Cafe,2,$$,Low,-86.474090,32.460040,136 W Main St,NULL,NULL,36067,13343611020,https://www.yelp.com/biz/uncle-micks-cajun-market-and-cafe-prattville?adjust_creative=2FlPnf5brM1FB2UTY6vGnQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2FlPnf5brM1FB2UTY6vGnQ,1,18704,2021-12-08 15:00:52.453,9999-12-31 00:00:00.000,1


In [76]:
select * from _production.DimBusinessCategoryBridge

(19 rows affected)

Total execution time: 00:00:00.023

BusinessKey,BusinessCategoryKey,BusinessID,CategoryID,ValidFrom,ValidTo,is_current
1,14,1,17,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,1
1,49,1,52,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,1
2,83,2,84,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,1
8,98,5,99,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,1
8,164,5,165,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,1
8,166,5,167,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,1
2,319,2,319,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,1
6,495,4,495,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,1
1,560,1,559,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,1
3,565,3,564,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,1


Here we can see that the first instance retained it's categories and now the next one that is current retains only one category as the previous connection was deleted before it was instantiated

Note: the first weeney hut junior doesn't show up since while it is active during active connections and active categories... specifically 

the connections are not altered nor any categories concerning it so it is bypassed instead of being included in the bridge here. No information is lost as the next active 

instance takes on any categories/connections left after the first instance

In [79]:
with bdg1 as (
    SELECT
        db1.BusinessKey,
        dbc1.BusinessCategoryKey,
        sbdg.BusinessID,
        sbdg.CategoryID,
        db1.ChainName,
        dbt_valid_from ValidFrom,
        CASE 
        WHEN dbt_valid_to is NULL THEN CAST('9999-12-31' as DATETIME)
        ELSE dbt_valid_to
        END AS ValidTo,
        CASE 
        WHEN dbt_valid_to is NULL THEN 1
        ELSE 0
        END AS is_current,
        db1.ValidFrom ValidFrom2,
        db1.ValidTo ValidTo2
    FROM "GourmandDWH"._Staging.dimbcb sbdg
    INNER JOIN "GourmandDWH"."_Production"."DimBusiness" db1 on sbdg.BusinessID=db1.BusinessSourceKey
    INNER JOIN "GourmandDWH"."_Production"."DimBusinessCategory" dbc1 on sbdg.CategoryID=dbc1.CategorySourceKey
  WHERE 
        (sbdg.dbt_valid_from BETWEEN db1.ValidFrom and db1.ValidTo
        OR sbdg.dbt_valid_to BETWEEN db1.ValidFrom and db1.ValidTo)

        AND (sbdg.dbt_valid_from BETWEEN dbc1.ValidFrom and dbc1.ValidTo
        OR sbdg.dbt_valid_to BETWEEN dbc1.ValidFrom and dbc1.ValidTo)

        and (DATEADD(Hour, DATEDIFF(Hour, 0, dbc1.ValidFrom), 0) 
        BETWEEN DATEADD(Hour, DATEDIFF(Hour, 0, db1.ValidFrom), 0) and DATEADD(Hour, DATEDIFF(Hour, 0, db1.ValidTo), 0)
        OR 
        DATEADD(Hour, DATEDIFF(Hour, 0, dbc1.ValidTo), 0) 
        BETWEEN DATEADD(Hour, DATEDIFF(Hour, 0, db1.ValidFrom), 0) and DATEADD(Hour, DATEDIFF(Hour, 0, db1.ValidTo), 0))

        and (DATEADD(Hour, DATEDIFF(Hour, 0, db1.ValidFrom), 0) 
        BETWEEN DATEADD(Hour, DATEDIFF(Hour, 0, dbc1.ValidFrom), 0) and DATEADD(Hour, DATEDIFF(Hour, 0, dbc1.ValidTo), 0)
        OR
        DATEADD(Hour, DATEDIFF(Hour, 0, db1.ValidTo), 0)
        BETWEEN DATEADD(Hour, DATEDIFF(Hour, 0, dbc1.ValidFrom), 0) and DATEADD(Hour, DATEDIFF(Hour, 0, dbc1.ValidTo), 0))
)
select * from bdg1

(19 rows affected)

Total execution time: 00:00:00.056

BusinessKey,BusinessCategoryKey,BusinessID,CategoryID,ChainName,ValidFrom,ValidTo,is_current,ValidFrom2,ValidTo2
1,49,1,52,Central,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,0,2021-12-06 15:02:55.797,9999-12-31 00:00:00.000
2,319,2,319,D' Road Cafe,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,0,2021-12-06 15:02:55.797,9999-12-31 00:00:00.000
3,565,3,564,LÃ¡Taste,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,0,2021-12-06 15:02:55.797,9999-12-31 00:00:00.000
4,495,4,495,Mrs Bâ€™s Home Cooking,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,0,2021-12-06 15:02:55.797,2021-12-09 20:53:54.800
6,495,4,495,Weeney Hut Junior2,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,0,2021-12-09 23:15:58.777,9999-12-31 00:00:00.000
7,164,5,165,Uncle Mick's Cajun Market & Cafe,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,0,2021-12-06 15:02:55.797,2021-12-08 15:00:52.453
8,164,5,165,Uncle Mick's Cajun Market & Cafe,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,0,2021-12-08 15:00:52.453,9999-12-31 00:00:00.000
1,14,1,17,Central,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,0,2021-12-06 15:02:55.797,9999-12-31 00:00:00.000
1,560,1,559,Central,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,0,2021-12-06 15:02:55.797,9999-12-31 00:00:00.000
2,83,2,84,D' Road Cafe,2021-12-07 10:59:52.203,9999-12-31 00:00:00.000,0,2021-12-06 15:02:55.797,9999-12-31 00:00:00.000


Now just some simples joins between some of the other tables

In [ ]:
select 
  fbh.BusinessKey,
  fbh.BusinessSourceKey,
  fbh.BusinessRating,
  fbh.ReviewCount,
  fbh.CloseDate,
  db.ChainName,
  db.PaymentLevelName,
  db.DistanceToCounty,
  dl.CityName,
  dd.TheDayName,
  dd.TheMonthName,
  dd.TheQuarter
FROM _Production.FactBusinessHolding fbh 
INNER JOIN _Production.DimBusiness db on fbh.BusinessKey=db.BusinessKey
INNER JOIN _Production.DimLocation dl on db.CitySourceKey=dl.CitySourceKey
INNER JOIN _Production.DimDate dd on fbh.CloseDate=dd.TheDate
ORDER BY ReviewCount DESC, BusinessRating DESC